# Haddock3 Protein-Protein Docking using HADDOCK3 with BioExcel Building Blocks (biobb)
**Based on the official [HADDOCK3 antibody-antigen modelling tutorial](https://www.bonvinlab.org/education/HADDOCK3/HADDOCK3-antibody-antigen/).**

***
This tutorial aims to illustrate the process of **proptein protein docking**, step by step, using **Haddock3** with the **BioExcel Building Blocks library (biobb)**. The particular systems used in this tutorial are the **Interleukin-1β (IL-1β) antigen** (PDB code 4I1B, https://doi.org/10.2210/pdb4I1B/pdb) and the **gevokizumab antibody** (PDB code 4G6K, https://doi.org/10.2210/pdb4G6K/pdb). The complex is also available under the PDB code 4G6M, https://doi.org/10.2210/pdb4G6M/pdb.

***

**Biobb modules** used:

* [biobb_io](https://github.com/bioexcel/biobb_io): Tools to fetch biomolecular data from public databases.
* [biobb_pdb_tools](https://github.com/bioexcel/biobb_pdb_tools): Swiss army knife for manipulating and editing PDB files. 
* [biobb_haddock](https://github.com/bioexcel/biobb_haddock): Module collection to compute information-driven flexible protein-protein docking.

**Auxiliary libraries** used:

* [jupyter](https://jupyter.org/): Free software, open standards, and web services for interactive computing across all programming languages.
* [nglview](http://nglviewer.org/#nglview): Jupyter/IPython widget to interactively view molecular structures and trajectories in notebooks.

### Conda Installation and Launch

```console
git clone https://github.com/bioexcel/biobb_wf_haddock.git
cd biobb_wf_haddock
conda env create -f conda_env/environment.yml
conda activate biobb_wf_haddock
jupyter-notebook biobb_wf_haddock/notebooks/biobb_wf_haddock_aa.ipynb
  ``` 

***
### Pipeline steps:
 1. [Input Parameters](#Input-parameters)
 2. [Preparing PDB files for docking](#Preparing-PDB-files-for-docking)
    - [Fetching PDB structures](#Fetching-PDB-structures)
    - [Preparing the antibody structure](#Preparing-the-antibody-structure)
    - [Preparing the antigen structure](#Preparing-the-antigen-structure)
    - [Preparing the reference structure](#Preparing-the-reference-structure)
 3. [Defining HADDOCK3 restraints](#Defining-HADDOCK3-restraints)
    - [Paratope Restraints](#Paratope-Restraints)
    - [Epitope Restraints](#Epitope-Restraints)
    - [HADDOCK3 passive restraints](#HADDOCK3-passive-restraints)
    - [HADDOCK3 ambiguous restraints](#HADDOCK3-ambiguous-restraints)
    - [Additional restraints for multi-chain proteins](#Additional-restraints-for-multi-chain-proteins)
 4. [Docking](#Docking)
    - [Create topology](#1.-Create-topology)
    - [Rigid Body sampling](#2.-Rigid-Body-sampling)
    - [1st CAPRI evaluation](#3.-1st-CAPRI-evaluation)
    - [Select Top structures](#4.-Select-Top-structures)
    - [Flexible Refinement](#5.-Flexible-Refinement)
    - [2nd CAPRI evaluation](#6.-2nd-CAPRI-evaluation)
    - [Energy minimization refinement](#7.-Energy-minimization-refinement)
    - [3rd CAPRI evaluation](#8.-3rd-CAPRI-evaluation)
    - [Clustering](#9.-Clustering)
    - [Selecting top clusters](#10.-Selecting-top-clusters)
    - [Final CAPRI evaluation](#11.-Final-CAPRI-evaluation)
    - [Contacts analysis](#12.-Contacts-analysis)
    - [Final Results](#13.-Final-Results)
 5. [Questions & Comments](#Questions-&-Comments) 

 
***

<table cellspacing="0" cellpadding="0" style="border-collapse: collapse; width: 100%;">
    <tr style="background: white;">
        <td style="text-align: center; vertical-align: middle; width: 50%;">
            <img src="https://bioexcel.eu/wp-content/uploads/2019/04/Bioexcell_logo_1080px_transp.png" alt="Bioexcel2 logo"
            title="Bioexcel logo" width="400" />        
        </td>
        <td style="text-align: center; vertical-align: middle; width: 50%;">
            <img src="imgs/HADDOCK3-logo.png" alt="HADDOCK3" style="max-width: 100%; height: auto; display: block; margin: auto;">
        </td>
    </tr>
</table>

***

<a class="anchor" name="box"></a>
## Initializing colab
The two cells below are used only in case this notebook is executed via **Google Colab**. Take into account that, for running conda on **Google Colab**, the **condacolab** library must be installed. As [explained here](https://pypi.org/project/condacolab/), the installation requires a **kernel restart**, so when running this notebook in **Google Colab**, don't run all cells until this **installation** is properly **finished** and the **kernel** has **restarted**.

In [1]:
# Only executed when using google colab
import sys
if 'google.colab' in sys.modules:
  import subprocess
  from pathlib import Path
  try:
    subprocess.run(["conda", "-V"], check=True)
  except FileNotFoundError:
    subprocess.run([sys.executable, "-m", "pip", "install", "condacolab"], check=True)
    import condacolab
    condacolab.install()
    # Clone repository
    repo_URL = "https://github.com/bioexcel/biobb_wf_haddock.git"
    repo_name = Path(repo_URL).name.split('.')[0]
    if not Path(repo_name).exists():
      subprocess.run(["mamba", "install", "-y", "git"], check=True)
      subprocess.run(["git", "clone", repo_URL], check=True)
      print("⏬ Repository properly cloned.")
    # Install environment
    print("⏳ Creating environment...")
    env_file_path = f"{repo_name}/conda_env/environment.yml"
    subprocess.run(["mamba", "env", "update", "-n", "base", "-f", env_file_path], check=True)
    print("🎨 Install NGLView dependencies...")
    subprocess.run(["mamba", "install", "-y", "-c", "conda-forge", "nglview==3.0.8", "ipywidgets=7.7.2"], check=True)
    print("👍 Conda environment successfully created and updated.")

In [2]:
# Enable widgets for colab
if 'google.colab' in sys.modules:
  from google.colab import output
  output.enable_custom_widget_manager()
  # Change working dir
  import os
  os.chdir("biobb_wf_haddock/biobb_wf_haddock/notebooks")
  print(f"📂 New working directory: {os.getcwd()}")

## Helper functions
Small **functions** used multiple times in the notebook and defined to make the code shorter and easier to follow.
- ***biobb_global_properties***: initializes the global properties dictionary used by all the BioBBs, with the paths for the log files (log.out, log.err). Avoids ending up with a folder full of log files.
- ***pdb_tools_pipeline***: launches a list of pdb_tools command line executions using the BioBB pdb_tools module.
- ***display_actpass***: loads a PDB file into NGLview widget and adds HADDOCK3 active and passive residues with surface representation.
- ***capri_visualization_models***: builds a dropdown to select specific models from the list of structures generated by HADDOCK. 
- ***superpose_ref***: superposes a PDB structure to a reference one.
- ***superpose_models***: superposes multiple PDB structures to a reference one and writes a multi-model PDB file.
- ***capri_visualization***: builds the NGL widget to show a 3D representation of the models generated by HADDOCK, with the possibility to select a specific model to represent. 
- ***open_results_mod***: opens a new tab in the web browser and loads a HADDOCK generated html file with summary results.

In [3]:
# Imports
import nglview as nv
import pytraj as pt
import ipywidgets
import webbrowser
import zipfile
import pandas
import glob
import os
import sys
import shutil

from Bio.PDB import PDBParser, PDBIO, CEAligner, Structure, Model, Chain
from IPython.display import display, Markdown
from jupyter_server.serverapp import list_running_servers
from biobb_common import biobb_global_properties

biobb_global_properties.update({
    'out_log_path': 'log/log.out',  # Define the folder and name for
    'err_log_path': 'log/log.err',  # output and error logs
    #'disable_sandbox': True,       # Disable the sandbox so input data is not copied
})

# All the inputs and outputs variables are also defined in the variables.py file
# in case you want to restart the notebook from another point
from variables import *

def pdb_tools_pipeline(inp_file, out_file, steps):
    """Helper function to concatenate calls to pdb_tools"""
    tmp_file = inp_file
    for step, props in steps:
        # Apply each step in the pipeline
        step(input_file_path  = tmp_file, 
             output_file_path = out_file, 
             properties       = props)
        tmp_file = 'tmp.pdb'
        os.rename(out_file, tmp_file)
    os.rename(tmp_file,out_file)

def display_actpass(pdb, actpass, opacity=1):
    with open(actpass, 'r') as file:
        actpass = file.read().splitlines()
        act_res = actpass[0].replace(' ', ', ')
        pas_res = actpass[1].replace(' ', ', ')

    # Load the PDB files
    view = nv.show_structure_file(pdb, default_representation=False)
    view.clear()
    view.add_cartoon(color='black')
    view.add_ball_and_stick(color='grey',opacity=opacity)
    view.add_surface(selection=f'not ( {pas_res}, {act_res} )', color='white', opacity=opacity)
    if act_res != '':
        view.add_surface(selection=f'{act_res}', color='red')
    if pas_res != '':
        view.add_surface(selection=f'{pas_res}', color='green', opacity=opacity)
    view.layout.width = '100%'
    view.center()
    return view

def step_backup(step):
    """Backup system in case you want to restart the workflow from a previous point."""
    backup = f'{out_path}/docking/step_backups/{step-1}'
    # Recover the backup
    if os.path.exists(backup):
        if os.path.exists(haddock_wf_data):
            shutil.rmtree(haddock_wf_data)
        shutil.copytree(backup, haddock_wf_data)
    # Backup the previous step
    if not os.path.exists(backup):
        shutil.copytree(haddock_wf_data, backup)
    # Remove the current step backup
    backup = f'{out_path}/docking/step_backups/{step}'
    if os.path.exists(backup):
        shutil.rmtree(backup)
        
def capri_visualization_models(tsv_dir,capri_out):

    models = []
    for i, m in enumerate(capri_out['model']):
        models.append(('Model Rank ' + str(i+1), os.path.normpath(os.path.join(tsv_dir, single_df['model'][i]))))
        
    mdsel = ipywidgets.Dropdown(
        options=models,
        description='Sel. model:',
        disabled=False,
    )
    
    # Dictionary of options in the dropdown
    options_dict = dict(mdsel.options)

    return mdsel

def superpose_ref(pdb_ref, pdb_to_sup, output_file, chain):

    # Parse the structures
    parser = PDBParser(QUIET=True)
    structure1 = parser.get_structure("pdb_ref", pdb_ref)
    structure2 = parser.get_structure("pdb_to_sup", pdb_to_sup)
    
    # Select only a portion of reference structure (e.g., Chain A )
    selected_residues = [res for res in structure1[0][chain]]
    
    # Create a new structure object with the selected residues
    selected_structure = Structure.Structure("selected_structure")
    model = Model.Model(0)
    chain = Chain.Chain("A")
    
    for res in selected_residues:
        chain.add(res)
    
    model.add(chain)
    selected_structure.add(model)    

    # Perform CE alignment using the selected region as reference
    aligner = CEAligner()
    aligner.set_reference(selected_structure)
    aligner.align(structure2)
    
    # Save structure2 to a PDB file
    io = PDBIO()
    io.set_structure(structure2)
    io.save(output_file)
    
def superpose_models(chain,input_path,output_file):
    
    pdb_files = sorted(glob.glob(f"{input_path}/*.pdb"))
    
    # Get all PDB files and sort them
    # Create a trajectory from the PDB files
    traj = pt.iterload(pdb_files, top=pdb_files[0])
    # Align the structures by chain and save the ensemble as a multi-model PDB
    pt.align(traj, ref=0, mask=f'::{chain}')
    traj.save(output_file, options="model", overwrite=True)

def capri_visualization(ensemble_A,ensemble_B,reference_A,reference_B,input_path,single_df,cluster_df):

    def on_dropdown_change(change):
        """Handle dropdown selection changes.
        From https://github.com/nglviewer/nglview/issues/765
        """
    
        if change['type'] == 'change' and change['name'] == 'value': 
            selected_file = change['new']
            if selected_file=='All':
                view1._remote_call('setSelection', target='compList', args=["*"], 
                   kwargs=dict(component_index=0))
                view2._remote_call('setSelection', target='compList', args=["*"], 
                   kwargs=dict(component_index=0))
            else:
                # Extract model number from the filename
                model_arr = selected_file.split('_')
                if (len(model_arr)) > 2:
                    cluster_num = int(model_arr[-3])
                    cluster_model_num = int(model_arr[-1])
                    model_num = (cluster_num * cluster_model_num) - 1
                else:
                    model_num = int(model_arr[-1]) - 1
                print(f"Selected model: {model_num}")
                # with open(f"{out_path}/example.txt", "w") as file:
                #     file.write(f"Hello: {model_num}")
                #     file.write(f"Dict: {dict(component_index=0)}")
                # Update the view with the selected model
                view1._remote_call('setSelection', target='compList', 
                                args=[f"/{model_num}"], 
                                kwargs=dict(component_index=0))
                # You can also update view2 if needed
                view2._remote_call('setSelection', target='compList', 
                                args=[f"/{model_num}"], 
                                kwargs=dict(component_index=0))

    # Create a dropdown widget
    pdb_files = sorted(glob.glob(f"{input_path}/*.pdb"))
    opts = ['All']
    opts.extend([pdb_file.split('/')[-1].split('.')[0] for pdb_file in pdb_files])
    mdsel = ipywidgets.Dropdown(
        options=opts,
        description='Sel. model:',
        disabled=False,
    )
    display(Markdown("#### Please select a model:"))
    display(mdsel)
    
    # Register the callback function
    mdsel.observe(on_dropdown_change, names='value')
    
    # Loading Ensemble aligned to chain A (Antibody)
    view1 = nv.show_structure_file(ensemble_A, default_representation=False)
    
    # Loading Ensemble aligned to chain B (Antigen)
    view2 = nv.show_structure_file(ensemble_B, default_representation=False)
    
    # Adding reference structure for comparison purposes
    view1.add_component(reference_A)
    view1.component_0.add_cartoon(color='cyan',opacity=.6)
    view2.add_component(reference_B)
    view2.component_0.add_cartoon(color='cyan',opacity=.6)
    
    # Colouring models aligned to chain A (Antibody: blue, Antigen: green) 
    view1.component_1.clear()
    view1.component_1.add_cartoon(selection=':A', color='purple')
    view1.component_1.add_cartoon(selection=':B', color='green')
    view1._remote_call('setSize', target='Widget', args=['500px','500px'])
    view1.layout.margin = "auto"
    view1.camera='orthographic'
    
    # Colouring models aligned to chain B (Antibody: blue, Antigen: green) 
    view2.component_1.clear()
    view2.component_1.add_cartoon(selection=':A', color='purple')
    view2.component_1.add_cartoon(selection=':B', color='green')
    view2._remote_call('setSize', target='Widget', args=['500px','500px'])
    view2.layout.margin = "auto"
    view2.camera='orthographic'
    
    # Display the viewer
    box = ipywidgets.HBox([view1, view2])
    display(Markdown("#### Generated models (left, aligned to chain A -Antibody-; right, aligned to chain B -Antigen-, cyan, reference)"))
    display(box)
    
    # Show CAPRI Evaluation values
    display(Markdown("#### CAPRI Evaluation values for Single Structure:"))
    display(Markdown("DockQ: incorrect (<0.23), acceptable (0.23-0.49), medium (0.49-0.80), and high (>=0.80)"))
    display(single_df.head(10))
    display(Markdown("#### CAPRI Evaluation values for Cluster-based output:"))
    display(cluster_df.head(10))

def open_results_mod(html_file):

    from IPython.display import HTML
    import shutil
    
    # Load the jupyter servers running
    servers = list(list_running_servers())
    
    if server := servers[0]:
        token = server.get("token")

        # Open the report file and modify accordingly
        with open(html_file, "r") as file:
            content = file.read()
        
        if 'id="banner"' not in content:
            banner = '<div id="banner" class="m-4" style="padding: 20px;font-weight: bold;background: #bde0ff;">To download a PDB file, please click the corresponding <b><i>Download</i></b> link with the <b><i>CTRL/COMMAND</i></b> key pressed</div><link href="https'
            content = content.replace('<link href="https', banner)
            content = content.replace("pdb.gz", f"pdb?token={token}")
            content = content.replace("..\\/", "..\\/..\\/")
        
            # Write the modified content back to the file
            with open(html_file, "w") as file:
                file.write(content)

        display(HTML(f'<a href="{html_file}" target="_blank">Click to open HADDOCK3 CAPRI evaluation report for this step</a>'))

    else:
        print("Could not determine the Jupyter Notebook port.")


/Users/gbayarri/miniconda3/envs/biobb_wf_haddock/lib/python3.12/site-packages/nglview/__init__.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


***
## Input parameters
**Input parameters** needed:
 - **antibody_pdb**: Antibody structure PDB code (in this case 4G6K)
 - **antigen_pdb**: Antigen structure PDB code (in this case 4I1B)
 - **complex_pdb**: Complex structure PDB code, if available (in this case 4G6M)
 - **out_path**: Output folder where resulting files will be generated

In [4]:
antibody_pdb = '4G6K'
antigen_pdb = '4I1B'
complex_pdb = '4G6M' 
out_path = './data/antibody'

Creating **output folder hierarchy**:
- ***out_path***
    - ***pre***: *Initial data* 
    - ***docking***: *Docking results*

In [5]:
#os.makedirs('data',exist_ok=True)
os.makedirs(out_path,exist_ok=True)
os.makedirs(f'{out_path}/pre',exist_ok=True)
os.makedirs(f'{out_path}/docking',exist_ok=True)

***
# Preparing PDB files for docking
 
Before initiating a docking process with **HADDOCK3**, the input PDB structures must meet a [**predefined set of requirements**](https://www.bonvinlab.org/haddock3-user-manual/structure_requirements.html#pdb-format). For example, each molecule should consist of a **single chain** with **non-overlapping residue numbering** within the same chain. Additionaly, **insertion codes** must be renumbered, an issue particularly problematic with antibodies, as often follow the Chothia numbering scheme and insertions created by this system (e.g. 82A, 82B, 82C) cannot be directly processed by **HADDOCK3**.

References: <br>

**ANARCI: antigen receptor numbering and receptor classification.**<br>
*Dunbar J, Deane CM.*<br>
*Bioinformatics, 2016;32(2):298-300.*<br>
*Available at: https://doi.org/10.1093/bioinformatics/btv552*
***

<div class="alert alert-block alert-info">
<b>IMPORTANT NOTE:</b> The preparation steps in this notebook were prepared for the specific systems used (<i>Interleukin-1β antigen, gevokizumab antibody</i>), which we know don't have issues such as <b>insertion codes</b> or <b>alternative locations</b>. Please consider using a more complete preparation pipeline for complex cases. Examples can be found in the <a href="https://www.bonvinlab.org/education/HADDOCK3/">HADDOCK3 official tutorials site</a>.   
</div>

***
## Fetching PDB structures
Downloading **PDB structures** from the [PDBe](https://www.ebi.ac.uk/pdbe/) database. Keeping only **standard residues** (removing heteroatoms). <br>
Alternatively, local PDB files can be used as starting structures. <br>
Downloading **three different files**: 
- **antibody**: Antibody protein structure
- **antigen**: Antigen protein structure
- **complex**: Antibody-antigen complex structure 

***
**Building Blocks** used:
 - [Pdb](https://biobb-io.readthedocs.io/en/latest/api.html#module-api.pdb) from **biobb_io.api.pdb**
***

In [6]:
# Downloading desired PDB files
# Import module
from biobb_io.api.pdb import pdb

## Antibody PDB

# Create properties dict and inputs/outputs
ab_pdb  = f'{out_path}/pre/{antibody_pdb}_0.pdb'
ab_prop = {
    'pdb_code': antibody_pdb,
    'filter': ['ATOM', 'TER', 'END'],
}

# Create and launch bb
pdb(output_pdb_path = ab_pdb,  
    properties = ab_prop)

## Antigen PDB

# Create properties dict and inputs/outputs
ag_pdb  = f'{out_path}/pre/{antigen_pdb}_0.pdb'
ag_prop = {
    'pdb_code': antigen_pdb,
    'filter': ['ATOM', 'TER', 'END']
}

# Create and launch bb
pdb(output_pdb_path = ag_pdb,  
    properties = ag_prop)

## Complex PDB

# Create properties dict and inputs/outputs
cx_pdb = f'{out_path}/pre/{complex_pdb}_0.pdb'
cx_prop = {
    'pdb_code': complex_pdb,
    'filter': ['ATOM', 'TER', 'END']
}

# Create and launch bb
pdb(output_pdb_path = cx_pdb,  
    properties = cx_prop)

2026-01-22 13:02:07,291 [MainThread  ] [INFO ]  Module: biobb_io.api.pdb Version: 5.2.0
2026-01-22 13:02:07,292 [MainThread  ] [INFO ]  Downloading 4g6k from: https://www.ebi.ac.uk/pdbe/entry-files/download/pdb4g6k.ent
2026-01-22 13:02:07,679 [MainThread  ] [INFO ]  Writting pdb to: ./data/antibody/pre/4G6K_0.pdb
2026-01-22 13:02:07,700 [MainThread  ] [INFO ]  Filtering lines NOT starting with one of these words: ['ATOM', 'TER', 'END']
2026-01-22 13:02:07,725 [MainThread  ] [INFO ]  
2026-01-22 13:02:07,729 [MainThread  ] [INFO ]  Module: biobb_io.api.pdb Version: 5.2.0
2026-01-22 13:02:07,730 [MainThread  ] [INFO ]  Downloading 4i1b from: https://www.ebi.ac.uk/pdbe/entry-files/download/pdb4i1b.ent
2026-01-22 13:02:07,986 [MainThread  ] [INFO ]  Writting pdb to: ./data/antibody/pre/4I1B_0.pdb
2026-01-22 13:02:07,988 [MainThread  ] [INFO ]  Filtering lines NOT starting with one of these words: ['ATOM', 'TER', 'END']
2026-01-22 13:02:07,992 [MainThread  ] [INFO ]  
2026-01-22 13:02:07,99

0

### Visualizing 3D structures
Visualizing the downloaded **PDB structures** using **NGLview**:  
- Left: **Antibody**. Heavy Chain: Blue; Light Chain: Red.
- Center: **Antigen** (Green)
- Right: **Complex**. Antibody: Heavy Chain: Blue; Light Chain: Red; Antigen: Green.

In [7]:
# Show structures:
# Antibody
view1 = nv.show_structure_file(ab_pdb)
view1._remote_call('setSize', target='Widget', args=['350px','400px'])
view1.clear_representations()
view1.add_representation(repr_type='cartoon', selection=':H', color='blue')
view1.add_representation(repr_type='cartoon', selection=':L', color='red')
view1.camera='orthographic'
# Antigen
view2 = nv.show_structure_file(ag_pdb)
view2._remote_call('setSize', target='Widget', args=['350px','400px'])
view2.clear_representations()
view2.add_representation(repr_type='cartoon', selection='protein', color='green')
view2.camera='orthographic'
# Complex
view3 = nv.show_structure_file(cx_pdb)
view3._remote_call('setSize', target='Widget', args=['350px','400px'])
view3.clear_representations()
view3.add_representation(repr_type='cartoon', selection=':H', color='blue')
view3.add_representation(repr_type='cartoon', selection=':L', color='red')
view3.add_representation(repr_type='cartoon', selection=':A', color='green')
view3.camera='orthographic'
ipywidgets.HBox([view1, view2, view3])

## Preparing the antibody structure

The **gevokizumab antibody**  needs to be processed to select only the **Heavy** and **Light** chains and combine both into one single file with a **unified chain** and **segment id** and **residue numbering** starting at 1. Additionaly, only the **variable fragment (Fv)** of the antibody (see image) will be selected and used, to shorten the structure size and thus reduce the computation time. 

<div style="text-align: center;">
    <img src="imgs/antigen.png" alt="antibody_image" width="400">
</div>

Please note that the PDB file used in this tutorial only contains the upper part of the **Heavy chains** (variable region). For a comprehensive example of a **complete antibody**, please refer to [5DK3 PDB](https://www.rcsb.org/structure/5DK3) (Pembrolizumab IgG4 antibody). 

***
**Building Blocks** used:
 - [biobb_pdb_tidy](https://biobb-pdb-tools.readthedocs.io/en/latest/pdb_tools.html#module-pdb_tools.biobb_pdb_tidy) from **biobb_pdb_tools.pdb_tools.biobb_pdb_tidy**
 - [biobb_pdb_selchain](https://biobb-pdb-tools.readthedocs.io/en/latest/pdb_tools.html#module-pdb_tools.biobb_pdb_selchain) from **biobb_pdb_tools.pdb_tools.biobb_pdb_selchain**
 - [biobb_pdb_delhetatm](https://biobb-pdb-tools.readthedocs.io/en/latest/pdb_tools.html#module-pdb_tools.biobb_pdb_delhetatm) from **biobb_pdb_tools.pdb_tools.biobb_pdb_delhetatm**
 - [biobb_pdb_fixinsert](https://biobb-pdb-tools.readthedocs.io/en/latest/pdb_tools.html#module-pdb_tools.biobb_pdb_fixinsert) from **biobb_pdb_tools.pdb_tools.biobb_pdb_fixinsert**
 - [biobb_pdb_selaltloc](https://biobb-pdb-tools.readthedocs.io/en/latest/pdb_tools.html#module-pdb_tools.biobb_pdb_selaltloc) from **biobb_pdb_tools.pdb_tools.biobb_pdb_selaltloc**
 - [biobb_pdb_keepcord](https://biobb-pdb-tools.readthedocs.io/en/latest/pdb_tools.html#module-pdb_tools.biobb_pdb_keepcord) from **biobb_pdb_tools.pdb_tools.biobb_pdb_keepcord**
 - [biobb_pdb_selres](https://biobb-pdb-tools.readthedocs.io/en/latest/pdb_tools.html#module-pdb_tools.biobb_pdb_selres) from **biobb_pdb_tools.pdb_tools.biobb_pdb_selres**
 - [biobb_pdb_reres](https://biobb-pdb-tools.readthedocs.io/en/latest/pdb_tools.html#module-pdb_tools.biobb_pdb_reres) from **biobb_pdb_tools.pdb_tools.biobb_pdb_reres**
 - [biobb_pdb_merge](https://biobb-pdb-tools.readthedocs.io/en/latest/pdb_tools.html#module-pdb_tools.biobb_pdb_merge) from **biobb_pdb_tools.pdb_tools.biobb_pdb_merge**
 - [biobb_pdb_chain](https://biobb-pdb-tools.readthedocs.io/en/latest/pdb_tools.html#module-pdb_tools.biobb_pdb_chain) from **biobb_pdb_tools.pdb_tools.biobb_pdb_chain**
 - [biobb_pdb_chainxseg](https://biobb-pdb-tools.readthedocs.io/en/latest/pdb_tools.html#module-pdb_tools.biobb_pdb_chainxseg) from **biobb_pdb_tools.pdb_tools.biobb_pdb_chainxseg**
***

#### Complete pdb_tools version from the official HADDOCK3 tutorial:
`pdb_tidy -strict 4G6K.pdb | pdb_selchain -H | pdb_delhetatm | pdb_fixinsert | pdb_selaltloc | pdb_keepcoord | pdb_selres -1:120 | pdb_tidy -strict > 4G6K_H.pdb`

### [Preparing the Antibody structure] Step 1: Select and extract protein regions/chains
**Extract chains** (Heavy: H, Light; L) and select specific residues from the **Variable Fragment (Fv)**.<br>
Two PDB files should be generated:
- *4G6K_H_reduced.pdb* (Heavy, FV)
- *4G6K_L_reduced.pdb* (Light, FV)

In [8]:
from biobb_pdb_tools.pdb_tools import  *

pdb_final = {}
for ch, sel in [('H',120),('L',107)]:
    
    # Create properties dict and inputs/outputs

    # CHAINS H (Heavy), L (Light)
    pdb_final[ch] = f'{out_path}/pre/{antibody_pdb}_{ch}_reduced.pdb'
    
    steps = [
        (biobb_pdb_tidy.biobb_pdb_tidy,           {'strict': True}),              # Adhere to the format specifications
        (biobb_pdb_selchain.biobb_pdb_selchain,   {'chains': ch}),                # Extract chain
        (biobb_pdb_delhetatm.biobb_pdb_delhetatm, {}),                            # Remove all HETATM records 
        (biobb_pdb_fixinsert.biobb_pdb_fixinsert, {}),                            # Delete insertion codes and shift residue numbering 
        (biobb_pdb_selaltloc.biobb_pdb_selaltloc, {}),                            # Select altloc labels (highest occupancy) 
        (biobb_pdb_keepcoord.biobb_pdb_keepcoord, {}),                            # Remove all non-coordinate records 
        (biobb_pdb_selres.biobb_pdb_selres,       {'selection': '1:'+str(sel)}),  # Select residues by their range
        (biobb_pdb_tidy.biobb_pdb_tidy,           {})                             # Adhere to the format specifications
    ]

    # Create and launch bb
    pdb_tools_pipeline(ab_pdb, pdb_final[ch], steps)

2026-01-22 13:05:01,364 [MainThread  ] [INFO ]  Module: biobb_pdb_tools.pdb_tools.biobb_pdb_tidy Version: 5.2.0
2026-01-22 13:05:01,366 [MainThread  ] [INFO ]  Directory successfully created: /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_c19e36d0-8eb6-4556-8585-2d5d083e42d3
2026-01-22 13:05:01,367 [MainThread  ] [INFO ]  Copy to stage: data/antibody/pre/4G6K_0.pdb --> sandbox_c19e36d0-8eb6-4556-8585-2d5d083e42d3
2026-01-22 13:05:01,369 [MainThread  ] [INFO ]  Appending optional boolean property
2026-01-22 13:05:01,370 [MainThread  ] [INFO ]  pdb_tidy -strict ./data/antibody/pre/4G6K_0.pdb > ./data/antibody/pre/4G6K_H_reduced.pdb
2026-01-22 13:05:01,370 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2026-01-22 13:05:01,372 [MainThread  ] [INFO ]  Launching command (it may take a while): pdb_tidy -strict ./data/antibody/pre/4G6K_0.pdb > ./data/antibody/pre/4G6K_H_reduced.pdb
2026-01-22 13:05:01,719 [Ma

### [Preparing the Antibody structure] Step 2: Merge regions into a single file
Merge **both chains** (Heavy, Light) into a single PDB file. <br>
- *4G6K_clean.pdb* (Heavy + Light chains, FV)

In [9]:
from biobb_pdb_tools.pdb_tools.biobb_pdb_merge import biobb_pdb_merge

# Join the generated PDB files in a single ZIP file 
zip_file_path = f'{out_path}/pre/{antibody_pdb}_HL.zip'
# Create a zip file and add the pdb_out file to it
with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    zipf.write(pdb_final['H'], arcname=f'{antibody_pdb}_H.pdb')
    zipf.write(pdb_final['L'], arcname=f'{antibody_pdb}_L.pdb')
    
# Create properties dict and inputs/outputs
complexFile = f'{out_path}/pre/{antibody_pdb}_clean_merge.pdb' 

# Create and launch bb
biobb_pdb_merge(input_file_path = zip_file_path,  
                output_file_path=complexFile)

2026-01-22 13:05:07,498 [MainThread  ] [INFO ]  Module: biobb_pdb_tools.pdb_tools.biobb_pdb_merge Version: 5.2.0
2026-01-22 13:05:07,502 [MainThread  ] [INFO ]  Directory successfully created: /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_f3bb64ce-8642-4c4c-a621-89d86576740b
2026-01-22 13:05:07,505 [MainThread  ] [INFO ]  Copy to stage: data/antibody/pre/4G6K_HL.zip --> sandbox_f3bb64ce-8642-4c4c-a621-89d86576740b
2026-01-22 13:05:07,512 [MainThread  ] [INFO ]  pdb_merge /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_f3bb64ce-8642-4c4c-a621-89d86576740b/4G6K_H.pdb /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_f3bb64ce-8642-4c4c-a621-89d86576740b/4G6K_L.pdb > ./data/antibody/pre/4G6K_clean_merge.pdb
2026-01-22 13:05:07,513 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2026-01-22 13:05:07,514 [MainThread  ] [I

0

### [Preparing the Antibody structure] Step 3: Prepare PDB file to meet HADDOCK3 requirements
Add **single chain id** to the whole structure, **renumber residue ids**, and add chain id in the **segment identifier**.<br>
- *4G6K_clean_seg.pdb* 

In [10]:
# Create properties dict and inputs/outputs
antibody_prep = f'{out_path}/pre/{antibody_pdb}_clean.pdb'

steps = [
    (biobb_pdb_reres.biobb_pdb_reres,         {'number': 1}),      # Renumber the residues starting from 1
    (biobb_pdb_chain.biobb_pdb_chain,         {'chain': 'A'}),     # Modify the chain identifier column 
    (biobb_pdb_chainxseg.biobb_pdb_chainxseg, {}),                 # Swap the segment identifier for the chain identifier
    (biobb_pdb_tidy.biobb_pdb_tidy,           {'strict': True}),   # Adhere to the format specifications
]

# Create and launch bb
pdb_tools_pipeline(complexFile, antibody_prep, steps)

2026-01-22 13:05:09,604 [MainThread  ] [INFO ]  Module: biobb_pdb_tools.pdb_tools.biobb_pdb_reres Version: 5.2.0
2026-01-22 13:05:09,606 [MainThread  ] [INFO ]  Directory successfully created: /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_562af0d0-892d-4e99-966c-012fc1d4c8cc
2026-01-22 13:05:09,608 [MainThread  ] [INFO ]  Copy to stage: data/antibody/pre/4G6K_clean_merge.pdb --> sandbox_562af0d0-892d-4e99-966c-012fc1d4c8cc
2026-01-22 13:05:09,612 [MainThread  ] [INFO ]  Appending optional boolean property
2026-01-22 13:05:09,612 [MainThread  ] [INFO ]  pdb_reres -1 /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_562af0d0-892d-4e99-966c-012fc1d4c8cc/4G6K_clean_merge.pdb > ./data/antibody/pre/4G6K_clean.pdb
2026-01-22 13:05:09,613 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2026-01-22 13:05:09,613 [MainThread  ] [INFO ]  Launching command (it may take a w

### [Preparing the Antibody structure] Step 4: Visualize final structure (antibody)

- Left: **Original antibody**, for comparison. Heavy Chain: Blue; Light Chain: Red.
- Right: **Prepared structure**, with only the variable domain (FV). Heavy Chain: Blue; Light Chain: Red.

In [11]:
# Show structures: Fv with and without contant domain
view1 = nv.show_structure_file(ab_pdb)
view1._remote_call('setSize', target='Widget', args=['500px','400px'])
view1.clear_representations()
view1.add_representation(repr_type='cartoon', selection=':H', color='blue')
view1.add_representation(repr_type='cartoon', selection=':L', color='red')
view1.add_representation(repr_type='surface', radius='.3', selection='1-120:H', color='blue')
view1.add_representation(repr_type='surface', radius='.3', selection='1-107:L', color='red')
view1.camera='orthographic'
view1
view2 = nv.show_structure_file(antibody_prep)
view2._remote_call('setSize', target='Widget', args=['500px','400px'])
view2.clear_representations()
view2.add_representation(repr_type='surface', radius='.3', selection='1-120:A', color='blue')
view2.add_representation(repr_type='surface', radius='.3', selection='121-228:A', color='red')
view2.camera='orthographic'
view2
ipywidgets.HBox([view1, view2])

## Preparing the antigen structure

The **Interleukin-1β (IL-1β) antigen** also needs to be processed to meet **HADDOCK3 requirements**. In particular, as each molecule given to **HADDOCK3** in a docking scenario must have a **unique chain id / segment id**, a new identifier ***-B-*** will be assigned.

***
**Building Blocks** used:
 - [biobb_pdb_tidy](https://biobb-pdb-tools.readthedocs.io/en/latest/pdb_tools.html#module-pdb_tools.biobb_pdb_tidy) from **biobb_pdb_tools.pdb_tools.biobb_pdb_tidy**
 - [biobb_pdb_delhetatm](https://biobb-pdb-tools.readthedocs.io/en/latest/pdb_tools.html#module-pdb_tools.biobb_pdb_delhetatm) from **biobb_pdb_tools.pdb_tools.biobb_pdb_delhetatm**
 - [biobb_pdb_selaltloc](https://biobb-pdb-tools.readthedocs.io/en/latest/pdb_tools.html#module-pdb_tools.biobb_pdb_selaltloc) from **biobb_pdb_tools.pdb_tools.biobb_pdb_selaltloc**
 - [biobb_pdb_keepcord](https://biobb-pdb-tools.readthedocs.io/en/latest/pdb_tools.html#module-pdb_tools.biobb_pdb_keepcord) from **biobb_pdb_tools.pdb_tools.biobb_pdb_keepcord**
 - [biobb_pdb_chain](https://biobb-pdb-tools.readthedocs.io/en/latest/pdb_tools.html#module-pdb_tools.biobb_pdb_chain) from **biobb_pdb_tools.pdb_tools.biobb_pdb_chain**
 - [biobb_pdb_chainxseg](https://biobb-pdb-tools.readthedocs.io/en/latest/pdb_tools.html#module-pdb_tools.biobb_pdb_chainxseg) from **biobb_pdb_tools.pdb_tools.biobb_pdb_chainxseg**

***

#### Complete pdb_tools version from the official HADDOCK3 tutorial:

`pdb_fetch 4I1B | pdb_tidy -strict | pdb_delhetatm | pdb_selaltloc | pdb_keepcoord | pdb_chain -B | pdb_chainxseg | pdb_tidy -strict > 4I1B_clean.pdb`

In [12]:
# Create properties dict and inputs/outputs
antigen_prep = f'{out_path}/pre/{antigen_pdb}_clean.pdb'

# Create and launch bb
steps = [
    (biobb_pdb_tidy.biobb_pdb_tidy,           {'strict': True}),   # Adhere to the format specifications
    (biobb_pdb_delhetatm.biobb_pdb_delhetatm, {}),                 # Remove all HETATM records 
    (biobb_pdb_selaltloc.biobb_pdb_selaltloc, {}),                 # Select altloc labels (highest occupancy) 
    (biobb_pdb_keepcoord.biobb_pdb_keepcoord, {}),                 # Remove all non-coordinate records 
    (biobb_pdb_chain.biobb_pdb_chain,         {'chain': 'B'}),     # Modify the chain identifier column 
    (biobb_pdb_chainxseg.biobb_pdb_chainxseg, {}),                 # Swap the segment identifier for the chain identifier
    (biobb_pdb_tidy.biobb_pdb_tidy,           {'strict': True})    # Adhere to the format specifications
]
pdb_tools_pipeline(ag_pdb, antigen_prep, steps)

2026-01-22 13:05:33,976 [MainThread  ] [INFO ]  Module: biobb_pdb_tools.pdb_tools.biobb_pdb_tidy Version: 5.2.0
2026-01-22 13:05:33,980 [MainThread  ] [INFO ]  Directory successfully created: /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_d1cd2a72-c2af-4c01-b3a7-285693aaee79
2026-01-22 13:05:33,983 [MainThread  ] [INFO ]  Copy to stage: data/antibody/pre/4I1B_0.pdb --> sandbox_d1cd2a72-c2af-4c01-b3a7-285693aaee79
2026-01-22 13:05:33,985 [MainThread  ] [INFO ]  Appending optional boolean property
2026-01-22 13:05:33,985 [MainThread  ] [INFO ]  pdb_tidy -strict ./data/antibody/pre/4I1B_0.pdb > ./data/antibody/pre/4I1B_clean.pdb
2026-01-22 13:05:33,986 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2026-01-22 13:05:33,987 [MainThread  ] [INFO ]  Launching command (it may take a while): pdb_tidy -strict ./data/antibody/pre/4I1B_0.pdb > ./data/antibody/pre/4I1B_clean.pdb
2026-01-22 13:05:34,054 [MainThread

### Visualize final structure (antigen)

- Left: **Original antigen**, for comparison. 
- Right: **Prepared structure**.

Note that the only difference in this case is the **chain / segment ids** (changed from ***-A-*** to ***-B-***).

In [13]:
view1 = nv.show_structure_file(ag_pdb)
view1._remote_call('setSize', target='Widget', args=['500px','400px'])
view1.clear_representations()
view1.add_representation(repr_type='cartoon', selection=':A', color='green')
view1.camera='orthographic'
view2 = nv.show_structure_file(antigen_prep)
view2._remote_call('setSize', target='Widget', args=['500px','400px'])
view2.clear_representations()
view2.add_representation(repr_type='cartoon', selection=':B', color='green')
view2.camera='orthographic'
print('Click on the proteins to see the changed chain name')
ipywidgets.HBox([view1, view2])

Click on the proteins to see the changed chain name


## Preparing the reference structure

Finally, since an experimentally solved structure of the **Antibody-Antigen** complex is available, we aim to compare the **intermediate results** of **HADDOCK3** with it. For this, the **reference PDB** should also be prepared. 

***
**Building Blocks** used:
 - [biobb_pdb_selchain](https://biobb-pdb-tools.readthedocs.io/en/latest/pdb_tools.html#module-pdb_tools.biobb_pdb_selchain) from **biobb_pdb_tools.pdb_tools.biobb_pdb_selchain**
 - [biobb_pdb_selres](https://biobb-pdb-tools.readthedocs.io/en/latest/pdb_tools.html#module-pdb_tools.biobb_pdb_selres) from **biobb_pdb_tools.pdb_tools.biobb_pdb_selres**
 - [biobb_pdb_reres](https://biobb-pdb-tools.readthedocs.io/en/latest/pdb_tools.html#module-pdb_tools.biobb_pdb_reres) from **biobb_pdb_tools.pdb_tools.biobb_pdb_reres**
 - [biobb_pdb_merge](https://biobb-pdb-tools.readthedocs.io/en/latest/pdb_tools.html#module-pdb_tools.biobb_pdb_merge) from **biobb_pdb_tools.pdb_tools.biobb_pdb_merge**
 - [biobb_pdb_chain](https://biobb-pdb-tools.readthedocs.io/en/latest/pdb_tools.html#module-pdb_tools.biobb_pdb_chain) from **biobb_pdb_tools.pdb_tools.biobb_pdb_chain**
 - [biobb_pdb_chainxseg](https://biobb-pdb-tools.readthedocs.io/en/latest/pdb_tools.html#module-pdb_tools.biobb_pdb_chainxseg) from **biobb_pdb_tools.pdb_tools.biobb_pdb_chainxseg**
 - [biobb_pdb_tidy](https://biobb-pdb-tools.readthedocs.io/en/latest/pdb_tools.html#module-pdb_tools.biobb_pdb_tidy) from **biobb_pdb_tools.pdb_tools.biobb_pdb_tidy**
***

### [Preparing the Reference structure] Step 1: Select and extract Heavy and Light antibody regions/chains from the complex
**Extract chains** (Heavy: H, Light; L) and select specific residues from the **Variable Domain (FV)**.<br>
Two PDB files should be generated:
- *4G6M_H_reduced.pdb* (Heavy, FV)
- *4G6M_L_reduced.pdb* (Light, FV)

In [14]:
# Create properties dict and inputs/outputs
pdb_ref = {}
pdb_in  = f'{out_path}/pre/{complex_pdb}_0.pdb'

for ch, sel in [('H',120),('L',107)]:
    pdb_ref[ch] = f'{out_path}/pre/{complex_pdb}_{ch}_reduced.pdb'
    steps = [
        (biobb_pdb_selchain.biobb_pdb_selchain,   {'chains': ch}),                # Extract chains
        (biobb_pdb_delhetatm.biobb_pdb_delhetatm, {}),                            # Remove all HETATM records
        (biobb_pdb_fixinsert.biobb_pdb_fixinsert, {}),                            # Delete insertion codes and shift residue numbering
        (biobb_pdb_selaltloc.biobb_pdb_selaltloc, {}),                            # Select altloc labels (highest occupancy) 
        (biobb_pdb_keepcoord.biobb_pdb_keepcoord, {}),                            # Remove all non-coordinate records 
        (biobb_pdb_selres.biobb_pdb_selres,       {'selection': '1:'+str(sel)}),  # Select residues by their range
        (biobb_pdb_tidy.biobb_pdb_tidy,           {})                             # Adhere to the format specifications
    ]

    # Create and launch bb
    pdb_tools_pipeline(pdb_in, pdb_ref[ch], steps)

2026-01-22 13:10:52,559 [MainThread  ] [INFO ]  Module: biobb_pdb_tools.pdb_tools.biobb_pdb_selchain Version: 5.2.0
2026-01-22 13:10:52,561 [MainThread  ] [INFO ]  Directory successfully created: /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_e6fc5a6b-09e8-469f-8b01-0cd35dae987e
2026-01-22 13:10:52,563 [MainThread  ] [INFO ]  Copy to stage: data/antibody/pre/4G6M_0.pdb --> sandbox_e6fc5a6b-09e8-469f-8b01-0cd35dae987e
2026-01-22 13:10:52,566 [MainThread  ] [INFO ]  Appending chains to select
2026-01-22 13:10:52,567 [MainThread  ] [INFO ]  pdb_selchain -H /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_e6fc5a6b-09e8-469f-8b01-0cd35dae987e/4G6M_0.pdb > ./data/antibody/pre/4G6M_H_reduced.pdb
2026-01-22 13:10:52,567 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2026-01-22 13:10:52,569 [MainThread  ] [INFO ]  Launching command (it may take a while): pdb_selchain

### [Preparing the Reference structure] Step 2: Merge Heavy and Light regions (from the complex) into a single file
Merge **both chains** (Heavy, Light) into a single PDB file. <br>
- *4G6M_antibody_clean.pdb* (Heavy + Light chains, FV)

In [15]:
# Join the generated PDB files in a single ZIP file 
zip_file_path = f'{out_path}/pre/{complex_pdb}_HL.zip'
# Create a zip file and add the pdb_out file to it
with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    zipf.write(pdb_ref['H'], arcname=f'{complex_pdb}_H.pdb')
    zipf.write(pdb_ref['L'], arcname=f'{complex_pdb}_L.pdb')
    
# Create properties dict and inputs/outputs
complexFile_ref_antibody = f'{out_path}/pre/{complex_pdb}_antibody_clean.pdb' 

# Create and launch bb
biobb_pdb_merge(input_file_path = zip_file_path,  
                output_file_path=complexFile_ref_antibody)

2026-01-22 13:10:55,340 [MainThread  ] [INFO ]  Module: biobb_pdb_tools.pdb_tools.biobb_pdb_merge Version: 5.2.0
2026-01-22 13:10:55,342 [MainThread  ] [INFO ]  Directory successfully created: /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_2e108b8d-5b40-4589-b116-a1fcc92862de
2026-01-22 13:10:55,344 [MainThread  ] [INFO ]  Copy to stage: data/antibody/pre/4G6M_HL.zip --> sandbox_2e108b8d-5b40-4589-b116-a1fcc92862de
2026-01-22 13:10:55,347 [MainThread  ] [INFO ]  pdb_merge /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_2e108b8d-5b40-4589-b116-a1fcc92862de/4G6M_H.pdb /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_2e108b8d-5b40-4589-b116-a1fcc92862de/4G6M_L.pdb > ./data/antibody/pre/4G6M_antibody_clean.pdb
2026-01-22 13:10:55,348 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2026-01-22 13:10:55,348 [MainThread  ]

0

### [Preparing the Reference structure] Step 3: Prepare the Antibody PDB file (from the complex) to match HADDOCK3 requirements
Add **single chain id** to the whole structure, **renumber residue ids**, and add chain id in the **segment identifier**, with the aim of having a **reference PDB** that can be directly compared with the **HADDOCK3** generated models.<br>
- *4G6M_clean_antibody_final.pdb* 

In [16]:
# Create properties dict and inputs/outputs
complexFile_ref_antibody_final = f'{out_path}/pre/{complex_pdb}_clean_antibody_final.pdb'

steps = [
    (biobb_pdb_reres.biobb_pdb_reres,         {'number': 1}),     # Renumber the residues starting from 1
    (biobb_pdb_chain.biobb_pdb_chain,         {'chain': 'A'}),    # Modify the chain identifier column 
    (biobb_pdb_chainxseg.biobb_pdb_chainxseg, {}),                # Swap the segment identifier for the chain identifier
    (biobb_pdb_tidy.biobb_pdb_tidy,           {'strict': True})   # Adhere to the format specifications
]

# Create and launch bb
pdb_tools_pipeline(complexFile_ref_antibody, complexFile_ref_antibody_final, steps)


2026-01-22 13:10:57,026 [MainThread  ] [INFO ]  Module: biobb_pdb_tools.pdb_tools.biobb_pdb_reres Version: 5.2.0
2026-01-22 13:10:57,029 [MainThread  ] [INFO ]  Directory successfully created: /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_0f4095ad-977b-4cf8-acaf-a2b3e332be1b
2026-01-22 13:10:57,033 [MainThread  ] [INFO ]  Copy to stage: data/antibody/pre/4G6M_antibody_clean.pdb --> sandbox_0f4095ad-977b-4cf8-acaf-a2b3e332be1b
2026-01-22 13:10:57,037 [MainThread  ] [INFO ]  Appending optional boolean property
2026-01-22 13:10:57,038 [MainThread  ] [INFO ]  pdb_reres -1 /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_0f4095ad-977b-4cf8-acaf-a2b3e332be1b/4G6M_antibody_clean.pdb > ./data/antibody/pre/4G6M_clean_antibody_final.pdb
2026-01-22 13:10:57,038 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2026-01-22 13:10:57,039 [MainThread  ] [INFO ]  Launching com

### [Preparing the Reference structure] Step 4: Prepare the Antigen PDB file (from the complex) to match HADDOCK3 requirements 
Add **single chain id** to the whole structure, **renumber residue ids**, and add chain id in the **segment identifier**, with the aim of having a **reference PDB** that can be directly compared with the **HADDOCK3** generated models.<br>
- *4G6M_clean_antigen_final.pdb* 

In [17]:
# Create properties dict and inputs/outputs
complexFile_ref_antigen_final = f'{out_path}/pre/{complex_pdb}_clean_antigen_final.pdb'

steps = [
    (biobb_pdb_tidy.biobb_pdb_tidy,           {'strict': True}),    # Adhere to the format specifications
    (biobb_pdb_selchain.biobb_pdb_selchain,   {'chains': 'A'}),     # Extract chains
    (biobb_pdb_chain.biobb_pdb_chain,         {'chain': 'B'}),      # Modify the chain identifier column 
    (biobb_pdb_chainxseg.biobb_pdb_chainxseg, {}),                  # Swap the segment identifier for the chain identifier
    (biobb_pdb_delhetatm.biobb_pdb_delhetatm, {}),                  # Remove all HETATM records
    (biobb_pdb_fixinsert.biobb_pdb_fixinsert, {}),                  # Delete insertion codes and shift residue numbering
    (biobb_pdb_selaltloc.biobb_pdb_selaltloc, {}),                  # Select altloc labels (highest occupancy) 
    (biobb_pdb_keepcoord.biobb_pdb_keepcoord, {}),                  # Remove all non-coordinate records 
    (biobb_pdb_tidy.biobb_pdb_tidy,           {'strict': True})     # Adhere to the format specifications
]
# Create and launch bb
pdb_tools_pipeline(cx_pdb, complexFile_ref_antigen_final, steps)

2026-01-22 13:10:58,626 [MainThread  ] [INFO ]  Module: biobb_pdb_tools.pdb_tools.biobb_pdb_tidy Version: 5.2.0
2026-01-22 13:10:58,628 [MainThread  ] [INFO ]  Directory successfully created: /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_c4bd467b-6ef5-49ab-aa6d-eac83f7d21b4
2026-01-22 13:10:58,630 [MainThread  ] [INFO ]  Copy to stage: data/antibody/pre/4G6M_0.pdb --> sandbox_c4bd467b-6ef5-49ab-aa6d-eac83f7d21b4
2026-01-22 13:10:58,632 [MainThread  ] [INFO ]  Appending optional boolean property
2026-01-22 13:10:58,633 [MainThread  ] [INFO ]  pdb_tidy -strict ./data/antibody/pre/4G6M_0.pdb > ./data/antibody/pre/4G6M_clean_antigen_final.pdb
2026-01-22 13:10:58,633 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2026-01-22 13:10:58,651 [MainThread  ] [INFO ]  Launching command (it may take a while): pdb_tidy -strict ./data/antibody/pre/4G6M_0.pdb > ./data/antibody/pre/4G6M_clean_antigen_final.pdb
2026-01

### [Preparing the Reference structure] Step 5: Merge Heavy + Light regions and Antigen (from the complex) into a single file
Merge **all chains** (Antibody Heavy chain, Antibody Light chain, Antigen) into a single PDB file. <br>
- *4G6M_clean.pdb* (Heavy + Light chains -FV- + Antigen)

In [18]:
## Join the generated PDB files in a single ZIP file ##

zip_file_path = f'{out_path}/pre/{complex_pdb}_HL_B.zip'
# Create a zip file and add the pdb_out file to it
with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    zipf.write(complexFile_ref_antibody_final, arcname=f'{complex_pdb}_antibody.pdb')
    zipf.write(complexFile_ref_antigen_final, arcname=f'{complex_pdb}_antigen.pdb')
    
# Create properties dict and inputs/outputs
complex_prep = f'{out_path}/pre/{complex_pdb}_clean.pdb' 

steps = [
    (biobb_pdb_merge,         {}),                                 # Merges several PDB files into one
    (biobb_pdb_tidy.biobb_pdb_tidy,           {'strict': True})    # Adhere to the format specifications
]

# Create and launch bb
pdb_tools_pipeline(zip_file_path, complex_prep, steps)

2026-01-22 13:11:00,883 [MainThread  ] [INFO ]  Module: biobb_pdb_tools.pdb_tools.biobb_pdb_merge Version: 5.2.0
2026-01-22 13:11:00,886 [MainThread  ] [INFO ]  Directory successfully created: /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_52645984-ba09-4dc9-9d7a-ed89fc07cc8c
2026-01-22 13:11:00,887 [MainThread  ] [INFO ]  Copy to stage: data/antibody/pre/4G6M_HL_B.zip --> sandbox_52645984-ba09-4dc9-9d7a-ed89fc07cc8c
2026-01-22 13:11:00,892 [MainThread  ] [INFO ]  pdb_merge /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_52645984-ba09-4dc9-9d7a-ed89fc07cc8c/4G6M_antibody.pdb /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_52645984-ba09-4dc9-9d7a-ed89fc07cc8c/4G6M_antigen.pdb > ./data/antibody/pre/4G6M_clean.pdb
2026-01-22 13:11:00,893 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2026-01-22 13:11:00,893 [MainThr

### [Preparing the Reference structure] Step 6: Visualize final structure (antibody-antigen complex)

Visualizing the processed **antibody-antigen complex** using **NGLview**:  
- Left: **Original Antibody-Antigen complex** for comparison. Heavy Chain: Green; Light Chain: Blue; Antigen: Red
- Right: **Processed Antibody-Antigen complex** with only the variable domain (FV). Heavy Chain: Green; Light Chain: Blue; Antigen: Red

In [19]:
# Show structures: antibody, antigen and complex
view1 = nv.show_structure_file(cx_pdb)
view1._remote_call('setSize', target='Widget', args=['500px','400px'])
view1.clear_representations()
view1.add_representation(repr_type='surface', radius='0.3', selection=':H', color='blue')
view1.add_representation(repr_type='surface', radius='0.3', selection=':L', color='red')
view1.add_representation(repr_type='surface', radius='0.3', selection=':A', color='green')
view1.camera='orthographic'
view2 = nv.show_structure_file(complex_prep)
view2._remote_call('setSize', target='Widget', args=['500px','400px'])
view2.clear_representations()
view2.add_representation(repr_type='surface', radius='0.3', selection='1-120:A', color='blue')
view2.add_representation(repr_type='surface', radius='0.3', selection='121-227:A', color='red')
view2.add_representation(repr_type='surface', radius='0.3', selection=':B', color='green')
view2.camera='orthographic'
ipywidgets.HBox([view1, view2])

# Defining HADDOCK3 restraints

The **HADDOCK3 protein-protein docking** tool integrates diverse sources of information, including experimental data, biochemical and biophysical insights, bioinformatics predictions, and prior knowledge, to **guide** and **refine** the docking process effectively. In this tutorial, knowledge of the **hypervariable loops** ([CDRs](https://en.wikipedia.org/wiki/Complementarity-determining_region)) on the **antibody**, and **epitope** information identified from **NMR experiments** for the **antigen** will be used to **guide the docking**.

The small part of the **antibody Fv region** that binds the **antigen** is called **paratope**. The part of the **antigen** that binds to an **antibody** is called **epitope** (see image, left). The **paratope** consists of six **highly flexible loops**, known as **complementarity-determining regions** (CDRs) or **hypervariable loops** whose sequence and conformation are altered to bind to different **antigens** (see image, right). 

Both regions are going to be used as **HADDOCK3 restrictions** to **guide** the **docking process**.

<table cellspacing="0" cellpadding="0" style="border-collapse: collapse; width: 100%;">
    <tr style="background: white;">
        <td style="text-align: center; vertical-align: middle; width: 25%;">
            <img src="imgs/EpitopeParatope.png" alt="Epitope/Paratope" style="max-width: 100%; height: auto; display: block; margin: auto;">
            <i>Paratope (Antibody) and Epitope (Antigen) representation<br> 
        </td>
        <td style="text-align: center; vertical-align: middle; width: 50%;">
            <img src="imgs/CDRs_reduced.png" alt="CDRs" style="max-width: 100%; height: auto; display: block; margin: auto;">
            <i>Paratope complementarity-determining regions (CDRs)<br> Figure represented with permissions from the official <a href="https://www.bonvinlab.org/education/HADDOCK3/HADDOCK3-antibody-antigen/#introduction">HADDOCK3 antibody-antigen tutorial</a></i>
        </td>
    </tr>
</table>

### Paratope Restraints

Developing new **vaccines** and **antibody therapeutics** is a lengthy process, often spanning several years. **Prediction** and understanding of the **paratope** region (**antibody binding site**) can **accelerate** this timeline and **lower the costs**. 
  
As a result, the research community has developed a wide range of tools to **predict paratope regions** (residues within **hypervariable loops** that play a key role in **antibody-antigen binding**) directly from **antibody sequences**.

This tutorial will work with a list of **paratope residues** obtained using the **ProABC-2 predictor** (see reference). **ProABC-2** uses a **convolutional neural network** to identify not only residues which are located in the **paratope region** but also the nature of interactions they are most likely involved in (hydrophobic or hydrophilic). 

**Paratope residues** used are those with either an **overall probability >= 0.4** or a **probability for hydrophobic or hydrophilic > 0.3**. Residue numbering corresponding to the processed file: *4G6K_clean.pdb*

References: <br>

**proABC-2: PRediction of AntiBody contacts v2 and its application to information-driven docking.**<br>
*F. Ambrosetti, T. H. Olsen, P. P. Olimpieri, B. Jiménez-García, E. Milanetti, P. Marcatilli, A. MJJ Bonvin*<br>
*Bioinformatics, 2020;36(20):5107-5108.*<br>
*Available at: https://doi.org/10.1093/bioinformatics/btaa644*<br>
*Source Code: https://github.com/haddocking/proabc-2*
***

In [20]:
# Antibody paratope residue list from proABC-2
paratope_sel = '31,32,33,34,35,52,54,55,56,100,101,102,103,104,105,106,151,152,169,170,173,211,212,213,214,216'

In [21]:
# Show antibody paratope
view = nv.show_structure_file(antibody_prep)
view._remote_call('setSize', target='Widget', args=['500px','500px'])
view.add_representation(repr_type='surface', selection=paratope_sel.replace(',', ', '), color='#a271a2')
centered_view = ipywidgets.HBox([view], layout=ipywidgets.Layout(justify_content="center"))
centered_view

### Epitope Restraints

The work describing the **crystal structures** deposited in the **PDB** for the **gevokizumab antibody** (PDB: 4G6K) and the **antibody-antigen** (Gevokizumab-Interleukin1β) complex (PDB: 4G6M) also comes with experimental **NMR chemical shift** titration experiments to map the **binding site** of the antibody on **Interleukin1β**. The residues affected by binding are listed in Table 5 of Blech et al. JMB 2013 (see references below). 

References: <br>

**One Target—Two Different Binding Modes: Structural Insights into Gevokizumab and Canakinumab Interactions to Interleukin-1β.**<br>
*M. Blech, D. Peter, P Fischer, M. M.T. Bauer, M. Hafner, M. Zeeb, H. Nar*<br>
*Journal of Molecular Biology, 2013;425(1):94-111.*<br>
*Available at: https://doi.org/10.1016/j.jmb.2012.09.021*
***

In [22]:
# Antigen Epitope residue list from Blech et al. JMB 2013, Table 5
epitope_sel  = '72,73,74,75,81,83,84,89,90,92,94,96,97,98,115,116,117'

In [28]:
# Show antigen epitope
view = nv.show_structure_file(antigen_prep)
view._remote_call('setSize', target='Widget', args=['500px','400px'])
view.add_representation(repr_type='surface', selection=epitope_sel.replace(',', ', '), color='#a9d18e')
centered_view = ipywidgets.HBox([view], layout=ipywidgets.Layout(justify_content="center"))
centered_view

### Visualize interfaces (paratope, epitope)

Visualizing the **predicted paratope** and the **described epitope** using **NGLview**:  
- Left: **predicted paratope** (light purple), with only the antibody variable domain (Fv). 
- Center: **described epitope** (light green), with the antigen.
- Right: **predicted paratope** (light purple) and  **described epitope** (light green) shown on the reference. **Antibody-Antigen complex** with only the variable domain (Fv). Antibody Fv: Blue; Antigen: Green.

In [29]:
# Show structures:
# Antibody
view1 = nv.show_structure_file(antibody_prep)
view1._remote_call('setSize', target='Widget', args=['350px','400px'])
view1.clear_representations()
view1.add_representation(repr_type='cartoon', selection=':A', color='purple')
view1.add_representation(repr_type='surface', ratio='0.3', selection=paratope_sel.replace(',', ', '), color='#a271a2')
view1.camera='orthographic'
# Antigen
view2 = nv.show_structure_file(antigen_prep)
view2._remote_call('setSize', target='Widget', args=['350px','400px'])
view2.clear_representations()
view2.add_representation(repr_type='cartoon', selection=':B', color='green')
view2.add_representation(repr_type='surface', ratio='0.3', selection=epitope_sel.replace(',', ', '), color='#a9d18e')
view2.camera='orthographic'
# Complex (for reference)
view3 = nv.show_structure_file(complex_prep)
view3._remote_call('setSize', target='Widget', args=['350px','400px'])
view3.clear_representations()
paratope_sel_ngl = ' '.join(f"{num}:A" for num in paratope_sel.split(','))
view3.add_representation(repr_type='cartoon', selection=':A', color='purple')
view3.add_representation(repr_type='surface', ratio='0.3', selection=paratope_sel_ngl, color='#a271a2')
epitope_sel_ngl = ' '.join(f"{num}:B" for num in epitope_sel.split(','))
view3.add_representation(repr_type='cartoon', selection=':B', color='green')
view3.add_representation(repr_type='surface', ratio='0.3', selection=epitope_sel_ngl, color='#a9d18e')
view3.camera='orthographic'
ipywidgets.HBox([view1, view2, view3])

### HADDOCK3 passive restraints

**Binding site interfaces** defined in the previous steps are tagged as **active residues** in **HADDOCK3**. Additionaly, **HADDOCK3** also gives the possibility to define **passive residues** to refine potentially **incomplete binding sites**. These residues are selected based on the **surface neighbors** of the **active residues**, and are included in the **interface definition**. However, they do not incur any **energetic penalty** if they are not part of the binding site in the final models. In contrast, **active residues** (typically identified or predicted as **key binding site residues**) are subject to an **energetic penalty** if they are not part of the binding site in the final models.

Here is the **HADDOCK3 definition** of **active** and **passive** residues:

**Active residues**: These residues are “forced” to be at the interface. If they are not part of the interface in the final models, an energetic penalty will be applied. The interface in this context is defined by the union of active and passive residues on the partner molecules.

**Passive residues**: These residues are expected to be at the interface. However, if they are not, no energetic penalty is applied.

In **HADDOCK3**, **passive** and **active residues** are defined in a **specific file** containing two lines:

- The first line corresponds to the list of **active residues** (numbers separated by spaces)
- The second line corresponds to the list of **passive residues** (numbers separated by spaces).

The file must always consist of two lines, but a line can be empty (e.g., if you do not want to define **active or passive residues** for one molecule). However, there must be at least one set of **active residues** defined for one of the molecules.

In this tutorial, a list of **passive residues** are defined for the **epitope region**, whereas no **passive residues** are going to be added to the **paratope region**, using only the **active residues** to guide the **docking process**. 

***
**Building Blocks** used:
 - [haddock3_passive_from_active](https://biobb-haddock.readthedocs.io/en/latest/haddock_restraints.html#module-haddock_restraints.haddock3_passive_from_active) from **biobb_haddock.haddock_restraints.haddock3_passive_from_active**
***

In [30]:
# Defining only active residues for antibody paratope region 
# Adding empty line as passive residues
ab_actpass = f'{out_path}/pre/{antibody_pdb}_actpass.txt' 
with open(ab_actpass, 'w') as f:
    f.write( paratope_sel.replace(',', ' ')+'\n\n')

In [31]:
# For the antigen, we will use the epitope selection as the active selection
# and some reidues around it as passsive

from biobb_haddock.haddock_restraints.haddock3_passive_from_active import haddock3_passive_from_active

# Create properties dict and inputs/outputs
ag_actpass = f'{out_path}/pre/{antigen_pdb}_actpass.txt'

prop = {'active_list' : epitope_sel}

# Create and launch bb
haddock3_passive_from_active( 
    input_pdb_path      = antigen_prep,
    output_actpass_path = ag_actpass,
    properties          = prop
)

2026-01-22 13:12:18,585 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock_restraints.haddock3_passive_from_active Version: 5.2.0
2026-01-22 13:12:18,586 [MainThread  ] [INFO ]  Directory successfully created: /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_573cc369-638e-4e3b-91d8-576ed771df82
2026-01-22 13:12:18,588 [MainThread  ] [INFO ]  Copy to stage: data/antibody/pre/4I1B_clean.pdb --> sandbox_573cc369-638e-4e3b-91d8-576ed771df82
2026-01-22 13:12:18,590 [MainThread  ] [INFO ]  Launching command (it may take a while): haddock3-restraints passive_from_active /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_573cc369-638e-4e3b-91d8-576ed771df82/4I1B_clean.pdb 72,73,74,75,81,83,84,89,90,92,94,96,97,98,115,116,117 > /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_573cc369-638e-4e3b-91d8-576ed771df82/4I1B_actpass.txt 2>&1
2026-01-22 13:12:19,882 [MainT

0

### Visualize active / passive residues (epitope)

Visualizing the defined **active** (red) and **passive** (green) residues for the **antigen epitope region** using **NGLview** 


In [32]:
display_actpass(antigen_prep, ag_actpass)

NGLWidget(layout=Layout(width='100%'))

### HADDOCK3 ambiguous restraints

Once **active** and **passive residues** are identified and defined for both molecules, they need to be transformed into CNS-formatted **Ambiguous Interaction Restraints** (AIR) files, so that they can be integrated in the **docking process**.  

**HADDOCK3** uses **CNS** as computational engine. A description of the format for the various restraint types supported by **HADDOCK3** can be found in the **Nature Protocol HADDOCK paper**, 2024, Box 1 (see reference below).

References: <br>

**The HADDOCK2.4 web server for integrative modeling of biomolecular complexes.**<br>
*R. Honorato et al.*<br>
*Nat. Protoc., 2024, 19, 3219–3241.*<br>
*Available at: https://doi.org/10.1038/s41596-024-01011-0*
***

**Building Blocks** used:
 - [haddock3_actpass_to_ambig](https://biobb-haddock.readthedocs.io/en/latest/haddock_restraints.html#module-haddock_restraints.haddock3_actpass_to_ambig) from **biobb_haddock.haddock_restraints.haddock3_actpass_to_ambig**
***

In [33]:
# Convert active/passive to ambiguous restraints
from biobb_haddock.haddock_restraints.haddock3_actpass_to_ambig import haddock3_actpass_to_ambig

# Create properties dict and inputs/outputs
complex_tbl = f'{out_path}/pre/ambig-paratope-NMR-epitope.tbl'

prop = {
    'segid_one': 'A', 
    'segid_two': 'B'
}

# Create and launch bb
haddock3_actpass_to_ambig( 
    input_actpass1_path=ab_actpass,
    input_actpass2_path=ag_actpass,    
    output_tbl_path=complex_tbl,
    properties = prop
)

2026-01-22 13:12:27,925 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock_restraints.haddock3_actpass_to_ambig Version: 5.2.0
2026-01-22 13:12:27,927 [MainThread  ] [INFO ]  Directory successfully created: /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_e56a400d-5806-4df8-9006-318786722442
2026-01-22 13:12:27,929 [MainThread  ] [INFO ]  Copy to stage: data/antibody/pre/4G6K_actpass.txt --> sandbox_e56a400d-5806-4df8-9006-318786722442
2026-01-22 13:12:27,931 [MainThread  ] [INFO ]  Copy to stage: data/antibody/pre/4I1B_actpass.txt --> sandbox_e56a400d-5806-4df8-9006-318786722442
2026-01-22 13:12:27,945 [MainThread  ] [INFO ]  Launching command (it may take a while): haddock3-restraints active_passive_to_ambig /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_e56a400d-5806-4df8-9006-318786722442/4G6K_actpass.txt /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/s

0

### Additional restraints for multi-chain proteins

As an **antibody** consists of two separate chains, it is important to define a few **distance restraints** to keep them together during the high temperature **flexible refinement** stage of **HADDOCK3** otherwise they might slightly drift appart. 

The following step generates **CA-CA distance restraints** with the exact distance measured between **randomly picked CA atoms pairs**, which will keep the two chains together.

***

**Building Blocks** used:
 - [haddock3_restrain_bodies](https://biobb-haddock.readthedocs.io/en/latest/haddock_restraints.html#module-haddock_restraints.haddock3_restrain_bodies) from **biobb_haddock.haddock_restraints.haddock3_restrain_bodies**
***

In [34]:
# Tie antibody chains together
from biobb_haddock.haddock_restraints.haddock3_restrain_bodies import haddock3_restrain_bodies

# Create properties dict and inputs/outputs
body_tbl = f'{out_path}/pre/antibody-unambig.tbl'

# Create and launch bb
haddock3_restrain_bodies( 
    input_structure_path=antibody_prep,
    output_tbl_path=body_tbl
)

2026-01-22 13:12:30,487 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock_restraints.haddock3_restrain_bodies Version: 5.2.0
2026-01-22 13:12:30,488 [MainThread  ] [INFO ]  Directory successfully created: /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_543bc5c0-23da-48d6-a57d-121ea6dc5aea
2026-01-22 13:12:30,490 [MainThread  ] [INFO ]  Copy to stage: data/antibody/pre/4G6K_clean.pdb --> sandbox_543bc5c0-23da-48d6-a57d-121ea6dc5aea
2026-01-22 13:12:30,492 [MainThread  ] [INFO ]  Launching command (it may take a while): haddock3-restraints restrain_bodies /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_543bc5c0-23da-48d6-a57d-121ea6dc5aea/4G6K_clean.pdb > /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_543bc5c0-23da-48d6-a57d-121ea6dc5aea/antibody-unambig.tbl 2>&1
2026-01-22 13:12:31,017 [MainThread  ] [INFO ]  Command 'haddock3-restraints restrain_bo

0

# Docking


**HADDOCK3** introduced a new **modular version** of the original **HADDOCK**, broken down in a catalogue of independent modules and then enriched with powerful analysis tools and third-party integrations (https://github.com/haddocking/haddock3, see reference below). In **HADDOCK3**, users have the freedom to configure **docking workflows** into functional pipelines by combining the different modules, thus adapting the **workflows** to their projects. **HADDOCK3** has therefore developed to truthfully work like a puzzle of many pieces (simulation modules) that users can combine freely. In the following cells, **BioBB building blocks** wrapping these independent modules are used to build a **HADDOCK3 docking workflow**, step by step. If you are interested in the **HADDOCK3 complete workflow** configuration file, please refer to the [official HADDOCK3 antibody-antigen tutorial section](https://www.bonvinlab.org/education/HADDOCK3/HADDOCK3-antibody-antigen/#haddock3-workflow-definition).  

<div align="center">
    <img src="imgs/HADDOCK3-workflow-scheme.png" alt="HADDOCK3 modularity" title="HADDOCK3 modularity" width="700"/>
    <br><i>HADDOCK3 workflow modularity<br>Figure represented with permissions from the official 
    <a href="https://www.bonvinlab.org/education/HADDOCK3/HADDOCK3-antibody-antigen/#introduction">HADDOCK3 antibody-antigen tutorial</a></i>
    <br><br>
</div>

The **docking workflow** consists of **14 steps**, outlined below. Since this is a **demonstration workflow**, and given the availability of reliable information on the **paratope** and **epitope**, the sampling steps have been reduced. This optimization minimizes computational cost and accelerates time-to-results while maintaining the accuracy of the generated models.

[**1. Create topology**](#1.-Create-topology): Generates the topologies for the CNS engine and builds missing atoms<br>
[**2. Rigid Body sampling**](#2.-Rigid-Body-sampling): Preforms rigid body energy minimization<br>
[**3. 1st CAPRI evaluation**](#3.-1st-CAPRI-evaluation): Calculates CAPRI metrics (i-RMSD, l-RMSD, Fnat, DockQ) with respect to the reference structure (complex)<br>
[**4. Select Top structures**](#4.-Select-Top-structures): Selects the top X models from the previous module<br>
[**5. Flexible Refinement**](#5.-Flexible-Refinement): Preforms semi-flexible refinement of the interface<br>
[**6. 2nd CAPRI evaluation**](#6.-2nd-CAPRI-evaluation): Calculates CAPRI metrics with the models generated in the previous step<br>
[**7. Energy minimization refinement**](#7.-Energy-minimization-refinement): Final refinement by energy minimization<br>
[**8. 3rd CAPRI evaluation**](#8.-3rd-CAPRI-evaluation): Calculates CAPRI metrics with the models generated in the previous step<br>
[**9. Clustering**](#9.-Clustering): Clustering of models based on the *Fraction of Common Contacts* (FCC)<br>
[**10. Selecting top clusters**](#10.-Selecting-top-clusters): Selects the top models of all clusters<br>
[**11. Final CAPRI evaluation**](#11.-Final-CAPRI-evaluation): Calculates CAPRI metrics with the models generated in the previous step<br>
[**12. Contacts analysis**](#12.-Contacts-analysis): Contacts analysis of intermolecular contacts<br>
[**13. Docking Results**](#13.-Docking-Results): Visualize a summary of the final models with statistics and energies<br>


To know more about the HADDOCK3 modules, please refer to the [**HADDOCK3 modules documentation**](https://www.bonvinlab.org/haddock3/modules/index.html)

References: <br>

**HADDOCK3: A modular and versatile platform for integrative modelling of biomolecular complexes.**<br>
*M. Giulini, V. Reys, J.M.C. Teixeira, B. Jiménez-García, R. V. Honorato, A. Kravchenko, X. Xu, R. Versini, A. Engel, S. Verhoeven, A. M.J.J. Bonvin*<br>
*BioArxiv, 2025*<br>
*Available at: https://doi.org/10.1101/2025.04.30.651432*
***

In [35]:
# Important variables to remember; those storing file names needed for the Docking process

# antibody_prep  --> Processed antibody PDB file
# antigen_prep --> Processed antigen PDB file
# complex_prep --> Processed antibody-antigen complex PDB file

# complex_tbl --> HADDOCK paratope/epitope restraints (AIR file)
# body_tbl --> HADDOCK multi-body restraints

***
### 1. Create topology

Create the **CNS all-atom topology**: CNS compatible parameters (.param) and topologies (.psf) for each of the input structures. Detects **missing atoms**, including **hydrogen atoms**, re-builds them when missing, builds and writes out **topologies** (psf) and **coordinates** (PDB) files.

This module is a **pre-requisite** to run any downstream steps and so is often used as the first module in a **HADDOCK3 workflow**.

***
**Building Blocks** used:
 - [topology](https://biobb-haddock.readthedocs.io/en/latest/haddock.html#module-haddock.topology) from **biobb_haddock.haddock.topology**
***
Corresponding **HADDOCK3 module**:
 - [topo_aa](https://www.bonvinlab.org/haddock3/modules/topology/haddock.modules.topology.topoaa.html) from **topology_modules**
***

In [36]:
from biobb_haddock.haddock.topology import topology
step_nb = 1

# Create properties dict and inputs/outputs
haddock_wf_data = f'{out_path}/docking/haddock_wf_data'
mol1_output_top_zip_path = f'{out_path}/docking/step_outputs/{step_nb}_top_mol1.zip'
mol2_output_top_zip_path = f'{out_path}/docking/step_outputs/{step_nb}_top_mol2.zip'

# Create and launch bb
topology(mol1_input_pdb_path        = antibody_prep,
         mol2_input_pdb_path        = antigen_prep,
         mol1_output_top_zip_path   = mol1_output_top_zip_path,
         mol2_output_top_zip_path   = mol2_output_top_zip_path,
         output_haddock_wf_data     = haddock_wf_data)

2026-01-22 13:12:40,673 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.topology Version: 5.2.0
2026-01-22 13:12:40,674 [MainThread  ] [INFO ]  Directory successfully created: /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_29ac5cb1-e6bb-4daf-80b5-4d95ba7f73fb
2026-01-22 13:12:40,676 [MainThread  ] [INFO ]  Copy to stage: data/antibody/pre/4G6K_clean.pdb --> sandbox_29ac5cb1-e6bb-4daf-80b5-4d95ba7f73fb
2026-01-22 13:12:40,677 [MainThread  ] [INFO ]  Copy to stage: data/antibody/pre/4I1B_clean.pdb --> sandbox_29ac5cb1-e6bb-4daf-80b5-4d95ba7f73fb
2026-01-22 13:12:40,679 [MainThread  ] [INFO ]  Launching command (it may take a while): haddock3 /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_29ac5cb1-e6bb-4daf-80b5-4d95ba7f73fb/149e63e3-ce09-4179-baf1-4e1880ee5284_haddock.cfg
2026-01-22 13:12:54,034 [MainThread  ] [INFO ]  Command 'haddock3 /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddo

0

***
### 2. Rigid Body sampling

**Randomization of orientations** and **rigid-body minimization**. Interacting partners are treated as **rigid bodies**, meaning that all geometrical parameters such as **bond lengths, bond angles, and dihedral angles** are **frozen**. The partners are first separated in space and randomly rotated around their respective centres of mass. Afterwards, the molecules are brought together by **rigid-body energy minimization** with rotations and translation as the only degrees of freedom.

The **driving force** for this **energy minimization** is the **energy function**, which consists of the **intermolecular van der Waals** and **electrostatic energy** terms and the **restraints** defined to **guide** the docking. The definition of those **restraints** is particularly important as they effectively **guide** the minimization process. For example, with a stringent set of **AIRs** or **unambiguous distance restraints**, the solutions of the minimization will **converge** much better and the **sampling** can be limited. In **ab-initio mode**, however, very diverse solutions will be obtained and the **sampling** should be **increased** to make sure to sample enough the possible **interaction space**.

In this case, **restraints** generated in the previous section (**epitope/paratope** and **multi-chain protein**) will be used to **guide** the **rigid body sampling**. 

***
**Building Blocks** used:
 - [rigid_body](https://biobb-haddock.readthedocs.io/en/latest/haddock.html#module-haddock.rigid_body) from **biobb_haddock.haddock.rigid_body**
***
Corresponding **HADDOCK3 module**:
 - [rigidbody](https://www.bonvinlab.org/haddock3/modules/sampling/haddock.modules.sampling.rigidbody.html) from **sampling_modules**
***

In [37]:
from biobb_haddock.haddock.rigid_body import rigid_body
step_nb = 2
step_backup(2)

# Create properties dict and inputs/outputs
docking_output_zip_path = f'{out_path}/docking/step_outputs/{step_nb}_docking.zip'

prop={
    'cfg': {
        'sampling': 10, # Reduced sampling (10 instead of the default of 1000)
        #'sampling': 50, # Reduced sampling (50 instead of the default of 1000)
        #'sampling': 100, # Reduced sampling (100 instead of the default of 1000)
        'clean': False   # Not compressing generated PDB files
    },
}

# Create and launch bb
rigid_body(input_haddock_wf_data         = haddock_wf_data,
           output_haddock_wf_data        = haddock_wf_data,
           docking_output_zip_path       = docking_output_zip_path,
           ambig_restraints_table_path   = complex_tbl,
           unambig_restraints_table_path = body_tbl,
           properties                    = prop
)

2026-01-22 13:13:27,154 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.rigid_body Version: 5.2.0
2026-01-22 13:13:27,155 [MainThread  ] [INFO ]  Directory successfully created: /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_e9c547ef-9b1e-459d-a221-80b6e5fc7268
2026-01-22 13:13:27,157 [MainThread  ] [INFO ]  Copy to stage: data/antibody/docking/haddock_wf_data --> sandbox_e9c547ef-9b1e-459d-a221-80b6e5fc7268
2026-01-22 13:13:27,163 [MainThread  ] [INFO ]  Copy to stage: data/antibody/pre/ambig-paratope-NMR-epitope.tbl --> sandbox_e9c547ef-9b1e-459d-a221-80b6e5fc7268
2026-01-22 13:13:27,165 [MainThread  ] [INFO ]  Copy to stage: data/antibody/pre/antibody-unambig.tbl --> sandbox_e9c547ef-9b1e-459d-a221-80b6e5fc7268
2026-01-22 13:13:27,166 [MainThread  ] [INFO ]  CFG from properties: sampling = 10
2026-01-22 13:13:27,167 [MainThread  ] [INFO ]  CFG from properties: clean = False
2026-01-22 13:13:27,167 [MainThread  ] [INFO ]  CFG from p

1

***
### 3. 1st CAPRI evaluation

**CAPRI (Critical Assessment of PRedicted Interactions)** is a community wide initiative for testing computational algorithms in **blind predictions** of experimentally determined 3D structures of **protein complexes**, the “targets”, provided to **CAPRI** prior to publication.

This step calculates **metrics** used during the **CAPRI evaluation process** (i-RMSD, l-RMSD, Fnat, DockQ) with respect to the **reference structure** (complex).

***
**Building Blocks** used:
 - [capri_eval](https://biobb-haddock.readthedocs.io/en/latest/haddock.html#module-haddock.capri_eval) from **biobb_haddock.haddock.capri_eval**
***
Corresponding **HADDOCK3 module**:
 - [caprieval](https://www.bonvinlab.org/haddock3/modules/analysis/haddock.modules.analysis.caprieval.html) from **analysis_modules**
***

In [38]:
from biobb_haddock.haddock.capri_eval import capri_eval
step_nb = 3
step_backup(3)
# Create properties dict and inputs/outputs
output_evaluation_zip_path = f'{out_path}/docking/step_outputs/{step_nb}_caprieval.zip'

# Create and launch bb
capri_eval(input_haddock_wf_data      = haddock_wf_data,
           output_haddock_wf_data     = haddock_wf_data,
           reference_pdb_path         = complex_prep,
           output_evaluation_zip_path = output_evaluation_zip_path)

2026-01-22 13:13:42,821 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.capri_eval Version: 5.2.0
2026-01-22 13:13:42,823 [MainThread  ] [INFO ]  Directory successfully created: /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_a98a11b0-ff67-4d29-bcf8-3b10111bcaca
2026-01-22 13:13:42,824 [MainThread  ] [INFO ]  Copy to stage: data/antibody/docking/haddock_wf_data --> sandbox_a98a11b0-ff67-4d29-bcf8-3b10111bcaca
2026-01-22 13:13:42,834 [MainThread  ] [INFO ]  Copy to stage: data/antibody/pre/4G6M_clean.pdb --> sandbox_a98a11b0-ff67-4d29-bcf8-3b10111bcaca
2026-01-22 13:13:42,838 [MainThread  ] [INFO ]  CFG: postprocess = True
2026-01-22 13:13:42,839 [MainThread  ] [INFO ]  Launching command (it may take a while): haddock3 /Users/gbayarri/projects/BioBB/tutorials/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_a98a11b0-ff67-4d29-bcf8-3b10111bcaca/83b5d8d4-0495-4d27-962c-991a9d59c44f_haddock.cfg --extend-run /Users/gbayarri/projects/BioB

1

In [39]:
# Looking at the cluster-based (clt) and single-structure (ss) CAPRI evaluation outputs
tsv_dir = haddock_wf_data + '/2_caprieval/'

# Load the cluster and single data into pandas DataFrames
cluster_df = pandas.read_csv(tsv_dir + 'capri_clt.tsv', sep='\t',comment='#')
single_df = pandas.read_csv(tsv_dir + 'capri_ss.tsv', sep='\t',comment='#')

# Align reference to models
model = os.path.normpath(os.path.join(tsv_dir, "../1_rigidbody/rigidbody_1.pdb"))

ref_sup_A = f"{out_path}/aligned_reference_A.pdb"
ref_sup_B = f"{out_path}/aligned_reference_B.pdb"
superpose_ref(model, complex_prep, ref_sup_A, "A")
superpose_ref(model, complex_prep, ref_sup_B, "B")

# Align generated models by chain
input_path = f'{haddock_wf_data}/1_rigidbody'
for chain in ["A","B"]:
    output_file = f"{out_path}/aligned_ensemble_{chain}.pdb"
    superpose_models(chain,input_path,output_file)

# Show results
aligned_ensemble_A = f"{out_path}/aligned_ensemble_A.pdb"
aligned_ensemble_B = f"{out_path}/aligned_ensemble_B.pdb"
capri_visualization(aligned_ensemble_A,aligned_ensemble_B,ref_sup_A,ref_sup_B,input_path,single_df,cluster_df)

FileNotFoundError: [Errno 2] No such file or directory: './data/antibody/docking/haddock_wf_data/2_caprieval/capri_clt.tsv'

In [ ]:
# Open HADDOCK CAPRI evaluation results summary (browser) 
capri_analysis_path = f'{haddock_wf_data}/analysis/2_caprieval_analysis/report.html'
open_results_mod(capri_analysis_path)

***
### 4. Select Top structures

Select a **number of structures** from the input models. By default, the selection is based on the **HADDOCK score** of the models. The number of models to be selected is defined by the ***select*** parameter. In the standard **HADDOCK protocol**, this number is **200**, which can be increased if more models should be refined.

***
**Building Blocks** used:
 - [sele_top](https://biobb-haddock.readthedocs.io/en/latest/haddock.html#module-haddock.sele_top) from **biobb_haddock.haddock.sele_top**
***
Corresponding **HADDOCK3 module**:
 - [seletop](https://www.bonvinlab.org/haddock3/modules/analysis/haddock.modules.analysis.seletop.html) from **analysis_modules**
***

In [ ]:
from biobb_haddock.haddock.sele_top import sele_top
step_nb = 4
step_backup(4)
# Create properties dict and inputs/outputs
output_selection_zip_path = f'{out_path}/docking/step_outputs/{step_nb}_selected.zip'

prop={
    'cfg': {
        'select': 8,  # Selection of the top 8 best scoring complexes (instead of the default of 200)
        #'select': 20,  # Selection of the top 20 best scoring complexes (instead of the default of 200)
    }
}

# Create and launch bb
sele_top(input_haddock_wf_data     = haddock_wf_data,
         output_haddock_wf_data    = haddock_wf_data,
         output_selection_zip_path = output_selection_zip_path,
         properties                = prop)

***
### 5. Flexible Refinement

**Flexible refinement** with **CNS**, a **semi-flexible simulated annealing** (SA) protocol based on molecular dynamics in **torsion angle** space.

This refinement consists of several stages:

- High temperature rigid body molecular dynamics
- Rigid body SA
- Semi-flexible SA with flexible side-chains at the interface
- Semi-flexible SA with fully flexible interface (both backbone and side-chains)

By default, only the **interface regions** are treated as **flexible**, automatically defined based on the **intermolecular contacts**. It is also possible to manually define the **semi-flexible regions**, and also define **fully flexible regions** that are allowed to move throughout the entire protocol from the high temperature rigid body molecular dynamics on. The **temperature** and **number of steps** for the various stages can be tuned.

***
**Building Blocks** used:
 - [flex_ref](https://biobb-haddock.readthedocs.io/en/latest/haddock.html#module-haddock.flex_ref) from **biobb_haddock.haddock.flex_ref**
***
Corresponding **HADDOCK3 module**:
 - [flexref](https://www.bonvinlab.org/haddock3/modules/refinement/haddock.modules.refinement.flexref.html) from **refinement_modules**
***

In [ ]:
from biobb_haddock.haddock.flex_ref import flex_ref
step_nb = 5
step_backup(5)
# Create properties dict and inputs/outputs
refinement_output_zip_path = f'{out_path}/docking/step_outputs/{step_nb}_flexref.zip'

prop={
    'cfg': {
        'tolerance' : 5,   # Failure tolerance percentage
        'clean': False,    # Not compressing generated PDB files
    },
}

# Create and launch bb
flex_ref(input_haddock_wf_data         = haddock_wf_data,
         output_haddock_wf_data        = haddock_wf_data,
         refinement_output_zip_path    = refinement_output_zip_path,
         ambig_restraints_table_path   = complex_tbl,
         unambig_restraints_table_path = body_tbl,
         properties                    = prop)

***
### 6. 2nd CAPRI evaluation

**CAPRI (Critical Assessment of PRedicted Interactions)** is a community wide initiative for testing computational algorithms in **blind predictions** of experimentally determined 3D structures of **protein complexes**, the “targets”, provided to **CAPRI** prior to publication.

This step calculates **metrics** used during the **CAPRI evaluation process** (i-RMSD, l-RMSD, Fnat, DockQ) with respect to the **reference structure** (complex).

***
**Building Blocks** used:
 - [capri_eval](https://biobb-haddock.readthedocs.io/en/latest/haddock.html#module-haddock.capri_eval) from **biobb_haddock.haddock.capri_eval**
***
Corresponding **HADDOCK3 module**:
 - [caprieval](https://www.bonvinlab.org/haddock3/modules/analysis/haddock.modules.analysis.caprieval.html) from **analysis_modules**
***

In [ ]:
from biobb_haddock.haddock.capri_eval import capri_eval
step_nb = 6
step_backup(6)
# Create properties dict and inputs/outputs
output_evaluation_zip_path2 = f'{out_path}/docking/step_outputs/{step_nb}_caprieval2.zip'

# Create and launch bb
capri_eval(input_haddock_wf_data      = haddock_wf_data,
           output_haddock_wf_data     = haddock_wf_data,
           reference_pdb_path         = complex_prep,
           output_evaluation_zip_path = output_evaluation_zip_path2)

In [ ]:
# Looking at the cluster-based (clt) and single-structure (ss) CAPRI evaluation outputs
tsv_dir = haddock_wf_data + '/5_caprieval/'

# Load the cluster and single data into pandas DataFrames
cluster_df = pandas.read_csv(tsv_dir + 'capri_clt.tsv', sep='\t',comment='#')
single_df = pandas.read_csv(tsv_dir + 'capri_ss.tsv', sep='\t',comment='#')

# Align reference to models
model = os.path.normpath(os.path.join(tsv_dir, "../4_flexref/flexref_1.pdb"))
ref_sup_A = f"{out_path}/aligned_reference_A.pdb"
ref_sup_B = f"{out_path}/aligned_reference_B.pdb"
superpose_ref(model, complex_prep, ref_sup_A, "A")
superpose_ref(model, complex_prep, ref_sup_B, "B")

# Align generated models by chain
input_path = f'{haddock_wf_data}/4_flexref'
for chain in ["A","B"]:
    output_file = f"{out_path}/aligned_ensemble_{chain}.pdb"
    superpose_models(chain,input_path,output_file)

# Show results
aligned_ensemble_A = f"{out_path}/aligned_ensemble_A.pdb"
aligned_ensemble_B = f"{out_path}/aligned_ensemble_B.pdb"
capri_visualization(aligned_ensemble_A,aligned_ensemble_B,ref_sup_A,ref_sup_B,input_path,single_df,cluster_df)

In [ ]:
# Open HADDOCK CAPRI evaluation results summary (browser) 
capri_analysis_path = f'{haddock_wf_data}/analysis/5_caprieval_analysis/report.html'
open_results_mod(capri_analysis_path)

***
### 7. Energy minimization refinement

**Energy minimization** refinement with **CNS**, refines the input complexes by **energy minimization** using **conjugate gradient** method.
**Coordinates** of the **energy minimized structures** are saved, and each complex is then evaluated using the **HADDOCK scoring function**.

***
**Building Blocks** used:
 - [em_ref](https://biobb-haddock.readthedocs.io/en/latest/haddock.html#module-haddock.em_ref) from **biobb_haddock.haddock.em_ref**
***
Corresponding **HADDOCK3 module**:
- [emref](https://www.bonvinlab.org/haddock3/modules/refinement/haddock.modules.refinement.emref.html) from **refinement.modules**

In [ ]:
from biobb_haddock.haddock.em_ref import em_ref
step_nb = 7
step_backup(7)
# Create properties dict and inputs/outputs
refinement_output_zip_path = f'{out_path}/docking/step_outputs/{step_nb}_emref.zip'

prop={
    'cfg': {
        'tolerance' : 5,   # Failure tolerance percentage
        'clean': False,  # Not compressing generated PDB files
    },
}

# Create and launch bb
em_ref(input_haddock_wf_data         = haddock_wf_data,
       output_haddock_wf_data        = haddock_wf_data,
       refinement_output_zip_path    = refinement_output_zip_path,
       ambig_restraints_table_path   = complex_tbl,
       unambig_restraints_table_path = body_tbl,
       properties                    = prop)

***
### 8. 3rd CAPRI evaluation

**CAPRI (Critical Assessment of PRedicted Interactions)** is a community wide initiative for testing computational algorithms in **blind predictions** of experimentally determined 3D structures of **protein complexes**, the “targets”, provided to **CAPRI** prior to publication.

This step calculates **metrics** used during the **CAPRI evaluation process** (i-RMSD, l-RMSD, Fnat, DockQ) with respect to the **reference structure** (complex).

***
**Building Blocks** used:
 - [capri_eval](https://biobb-haddock.readthedocs.io/en/latest/haddock.html#module-haddock.capri_eval) from **biobb_haddock.haddock.capri_eval**
***
Corresponding **HADDOCK3 module**:
 - [caprieval](https://www.bonvinlab.org/haddock3/modules/analysis/haddock.modules.analysis.caprieval.html) from **analysis_modules**
***

In [ ]:
from biobb_haddock.haddock.capri_eval import capri_eval
step_nb = 8
step_backup(8)
# Create properties dict and inputs/outputs
output_evaluation_zip_path3 = f'{out_path}/docking/step_outputs/{step_nb}_caprieval3.zip'

# Create and launch bb
capri_eval(input_haddock_wf_data      = haddock_wf_data,
           output_haddock_wf_data     = haddock_wf_data,
           reference_pdb_path         = complex_prep,
           output_evaluation_zip_path = output_evaluation_zip_path3)

In [ ]:
# Looking at the cluster-based (clt) and single-structure (ss) CAPRI evaluation outputs
tsv_dir = haddock_wf_data + '/7_caprieval/'

# Load the cluster and single data into pandas DataFrames
cluster_df = pandas.read_csv(tsv_dir + 'capri_clt.tsv', sep='\t',comment='#')
single_df = pandas.read_csv(tsv_dir + 'capri_ss.tsv', sep='\t',comment='#')

# Align reference to models
model = os.path.normpath(os.path.join(tsv_dir, "../6_emref/emref_1.pdb"))
ref_sup_A = f"{out_path}/aligned_reference_A.pdb"
ref_sup_B = f"{out_path}/aligned_reference_B.pdb"
superpose_ref(model, complex_prep, ref_sup_A, "A")
superpose_ref(model, complex_prep, ref_sup_B, "B")

# Align generated models by chain
input_path = f'{haddock_wf_data}/6_emref'
for chain in ["A","B"]:
    output_file = f"{out_path}/aligned_ensemble_{chain}.pdb"
    superpose_models(chain,input_path,output_file)

# Show results
aligned_ensemble_A = f"{out_path}/aligned_ensemble_A.pdb"
aligned_ensemble_B = f"{out_path}/aligned_ensemble_B.pdb"
capri_visualization(aligned_ensemble_A,aligned_ensemble_B,ref_sup_A,ref_sup_B,input_path,single_df,cluster_df)

In [ ]:
# Open HADDOCK CAPRI evaluation results summary (browser) 
capri_analysis_path = f'{haddock_wf_data}/analysis/7_caprieval_analysis/report.html'
open_results_mod(capri_analysis_path)

***
### 9. Clustering 

**Cluster** models with the *Fraction of Common Contacts* (FCC) method. The module takes the models generated in the previous step and calculates the **contacts** between them. Then, calculates the **FCC matrix** and **clusters** the models based on the **calculated contacts**.

References: <br>

**Clustering biomolecular complexes by residue contacts similarity.**<br>
*J.P.G.L.M. Rodrigues, M. Trellet, C. Schmitz, P. Kastritis, E. Karaca, A.S.J. Melquiond, A.M.J.J. Bonvin.*<br>
*Proteins, 2012, 80(7),1810-7*<br>
*Available at: https://doi.org/10.1002/prot.24078*
***
**Building Blocks** used:
 - [clust_fcc](https://biobb-haddock.readthedocs.io/en/latest/haddock.html#module-haddock.clust_fcc) from **biobb_haddock.haddock.clust_fcc**
***
Corresponding **HADDOCK3 module**:
 - [clustfcc](https://www.bonvinlab.org/haddock3/modules/analysis/haddock.modules.analysis.clustfcc.html) from **analysis_modules**
***


In [ ]:
from biobb_haddock.haddock.clust_fcc import clust_fcc
step_nb = 9
step_backup(9)
# Create properties dict and inputs/outputs
output_cluster_zip_path = f'{out_path}/docking/step_outputs/{step_nb}_clustfcc.zip'

prop={
    'cfg': {
        'plot_matrix' : True,   # Plot matrix of members
    }
}

# Create and launch bb
clust_fcc(input_haddock_wf_data   = haddock_wf_data,
          output_haddock_wf_data  = haddock_wf_data,
          output_cluster_zip_path = output_cluster_zip_path,
          properties              = prop)

***
### 10. Selecting top clusters 

Select **models** from the **top clusters**. The selection is based on the **score** of the models within the clusters.

In the standard **HADDOCK** analysis, the **top 4 models** of the **top 10 clusters** are shown.

***
**Building Blocks** used:
 - [sele_top_clusts](https://biobb-haddock.readthedocs.io/en/latest/haddock.html#module-haddock.sele_top_clusts) from **biobb_haddock.haddock.sele_top_clusts**
***
Corresponding **HADDOCK3 module**:
 - [seletopclusts](https://www.bonvinlab.org/haddock3/modules/analysis/haddock.modules.analysis.seletopclusts.html) from **analysis_modules**
***

In [ ]:
from biobb_haddock.haddock.sele_top_clusts import sele_top_clusts
step_nb = 10
step_backup(10)
# Create properties dict and inputs/outputs
output_seletopclusts_zip_path = f'{out_path}/docking/step_outputs/{step_nb}_seletopclusts.zip'

prop={
    'cfg': {
        'top_models': 4,   # Selection of the top 4 best scoring complexes from each cluster
        'clean': False     # Not compressing generated PDB files
    },
}

# Create and launch bb
sele_top_clusts(input_haddock_wf_data     = haddock_wf_data,
                output_haddock_wf_data    = haddock_wf_data,
                output_selection_zip_path = output_seletopclusts_zip_path,
                properties                = prop)

***
### 11. Final CAPRI evaluation

**CAPRI (Critical Assessment of PRedicted Interactions)** is a community wide initiative for testing computational algorithms in **blind predictions** of experimentally determined 3D structures of **protein complexes**, the “targets”, provided to **CAPRI** prior to publication.

This step calculates **metrics** used during the **CAPRI evaluation process** (i-RMSD, l-RMSD, Fnat, DockQ) with respect to the **reference structure** (complex).

***
**Building Blocks** used:
 - [capri_eval](https://biobb-haddock.readthedocs.io/en/latest/haddock.html#module-haddock.capri_eval) from **biobb_haddock.haddock.capri_eval**
***
Corresponding **HADDOCK3 module**:
 - [caprieval](https://www.bonvinlab.org/haddock3/modules/analysis/haddock.modules.analysis.caprieval.html) from **analysis_modules**
***

In [ ]:
from biobb_haddock.haddock.capri_eval import capri_eval
step_nb = 11
step_backup(11)
# Create properties dict and inputs/outputs
output_evaluation_zip_path4 = f'{out_path}/docking/step_outputs/{step_nb}_caprieval4.zip'

# Create and launch bb
capri_eval(input_haddock_wf_data      = haddock_wf_data,
           output_haddock_wf_data     = haddock_wf_data,
           reference_pdb_path         = complex_prep,
           output_evaluation_zip_path = output_evaluation_zip_path4)

In [ ]:
# Looking at the cluster-based (clt) and single-structure (ss) CAPRI evaluation outputs
tsv_dir = haddock_wf_data + '/10_caprieval/'

# Load the cluster and single data into pandas DataFrames
cluster_df = pandas.read_csv(tsv_dir + 'capri_clt.tsv', sep='\t',comment='#')
single_df = pandas.read_csv(tsv_dir + 'capri_ss.tsv', sep='\t',comment='#')

# Align reference to models
model = os.path.normpath(os.path.join(tsv_dir, "../09_seletopclusts/cluster_1_model_1.pdb"))
ref_sup_A = f"{out_path}/aligned_reference_A.pdb"
ref_sup_B = f"{out_path}/aligned_reference_B.pdb"
superpose_ref(model, complex_prep, ref_sup_A, "A")
superpose_ref(model, complex_prep, ref_sup_B, "B")

# Align generated models by chain
input_path = f'{haddock_wf_data}/09_seletopclusts'
for chain in ["A","B"]:
    output_file = f"{out_path}/aligned_ensemble_{chain}.pdb"
    superpose_models(chain,input_path,output_file)

# Show results
aligned_ensemble_A = f"{out_path}/aligned_ensemble_A.pdb"
aligned_ensemble_B = f"{out_path}/aligned_ensemble_B.pdb"
capri_visualization(aligned_ensemble_A,aligned_ensemble_B,ref_sup_A,ref_sup_B,input_path,single_df,cluster_df)

In [ ]:
# Open HADDOCK CAPRI evaluation results summary (browser) 
capri_analysis_path = f'{haddock_wf_data}/analysis/10_caprieval_analysis/report.html'
open_results_mod(capri_analysis_path)

***
### 12. Contacts analysis

Compute **contacts** between chains in complexes, generating **heatmaps** and **chordcharts** of the **contacts** observed in the input complexes. If complexes are **clustered**, the analysis of contacts will be performed based on **all structures** from each **cluster**.

**Heatmaps** are describing the **probability of contacts** (<5A) between two residues (both intramolecular and intermolecular).

**Chordcharts** are describing only **intermolecular contacts in circles**, connecting with chords the two residues that are contacting.

***
**Building Blocks** used:
 - [contact_map](https://biobb-haddock.readthedocs.io/en/latest/haddock.html#module-haddock.contact_map) from **biobb_haddock.haddock.contact_map**
***
Corresponding **HADDOCK3 module**:
 - [contactmap](https://www.bonvinlab.org/haddock3/modules/analysis/haddock.modules.analysis.contactmap.html) from **analysis_modules**
***

In [ ]:
from biobb_haddock.haddock.contact_map import contact_map
step_nb = 12
step_backup(12)
# Create properties dict and inputs/outputs
output_contactmap_zip_path = f'{out_path}/docking/step_outputs/{step_nb}_contact_map.zip'

# Create and launch bb
contact_map(input_haddock_wf_data      = haddock_wf_data,
            output_haddock_wf_data     = haddock_wf_data,
            output_contactmap_zip_path = output_contactmap_zip_path)

In [ ]:
htmls = sorted(glob.glob(haddock_wf_data+'/11_contactmap/cluster*.html'))

contacts = []
for i, m in enumerate(htmls):
    contacts.append(m.split('/')[-1])
    
contact_sel = ipywidgets.Dropdown(
    options=contacts,
    description='Sel. model:',
    label=None,
)
def on_dropdown_change(change):
    """Handle dropdown selection changes.
    From https://github.com/nglviewer/nglview/issues/765
    """
    if change['type'] == 'change' and change['name'] == 'value': 
        contact_map_path = haddock_wf_data+f'/11_contactmap/{change["new"]}'
        print(contact_map_path)
        webbrowser.open(contact_map_path)
        
# Register the callback function
contact_sel.observe(on_dropdown_change, names='value')
contact_sel

***
### 13. Final Results

**Final results** of the **docking process** contain various steps of the defined **workflow** numbered sequentially starting at 0, e.g.:

     00_topoaa/
     01_rigidbody/
     02_caprieval/

In addition, there are four **additional directories** and a **log file**:

- ***analysis directory***: contains various plots to visualize the results for each caprieval step and a general report (report.html) that provides all statistics with various plots (to be opened in your preferred web browser)
- ***data directory***: contains the input data (PDB and restraint files) for the various modules, as well as an input workflow (in configurations directory)
- ***toppar directory***: contains the force field topology and parameter files 
- ***traceback directory***: contains traceback.tsv, which links all models to see which model originates from which throughout all steps of the workflow.
- ***log file***: text file with information about the docking process and duration of the run

Each ***sampling/refinement/selection*** module will contain **PDB files**. For example, the 09_seletopclusts directory contains the **selected models** from each cluster. The **clusters** in that directory are numbered based on their rank, i.e. cluster_1 refers to the top-ranked cluster. Information about the origin of these files can be found in that directory in the seletopclusts.txt file.

***

In [ ]:
# Listing directories of the final results
!ls {haddock_wf_data}

In [ ]:
# Listing -seletopclust- directory content (Note the generated PDB models)
!ls {haddock_wf_data}/09_seletopclusts

In [ ]:
# Listing ANALYSES directories of the final results
!ls {haddock_wf_data}/analysis

In [ ]:
# Listing -seletopclusts- directory content of the final ANALYSES results
!ls {haddock_wf_data}/analysis/7_caprieval_analysis 

***

## Questions & Comments

Questions, issues, suggestions and comments are really welcome!

* GitHub issues:
    * [https://github.com/bioexcel/biobb](https://github.com/bioexcel/biobb)

* BioExcel forum:
    * [https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library](https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library)
